# Evaluator Pipeline Notebook

**Clinical Authorization Determinations using a MedHelm Approach**

In this notebook we demonstrate an evaluator pipeline that:

1. **Establishes a Clinical Context:**  
   We apply a clinical approach via the MedHelm framework. This approach helps us to evaluate both prior authorization determinations and the clinical rationale behind them by systematically assessing patient, physician, and clinical information.

2. **Generates an LLM Response:**  
   We use the `AutoDeterminationEvaluator`’s `generate_responses` method to obtain a determination from an LLM based on the provided context.

3. **Handles Incomplete Context (Failure Scenario):**  
   We simulate a case where critical details (e.g., clinical or physical information) are omitted and the evaluator returns an unsatisfactory or failed determination.

4. **Uses an IaC Framework:**  
   Instead of ad hoc scripting, our configuration (e.g. test cases defined in YAML) is managed as code, allowing us to spin up or update the evaluation environment dynamically.

5. **Enforces Benchmarks with PyTest:**  
   We include tests (using PyTest) to guarantee that each evaluation case meets specified metric thresholds.

6. **Evaluator Discussion:**  
   Our evaluator setup leverages components such as the **FuzzyEvaluator** (which computes semantic similarity, e.g. using fuzzy matching techniques) and the **FactualCorrectnessEvaluator** (which checks the factual integrity of the clinical rationale).

_Below, we demonstrate each phase of the pipeline with code examples and commentary._

---

## 1. Clinical Context & the MedHelm Approach

The MedHelm approach emphasizes a thorough review of clinical, physician, and policy contexts:

- **Patient Information:** Name, DOB, address, etc.
- **Physician Information:** Specialty and contact details.
- **Clinical Information:** Diagnosis details, lab results, treatment history, and clinical reasoning.

In our evaluator pipeline, we load these details from YAML test cases and use them to generate LLM responses. This approach helps ensure that any prior authorization determination (or its clinical rationale) is based on rich, structured data.

We now move to the evaluator pipeline code that uses these contexts.

In [1]:
from dotenv import load_dotenv

from src.aifoundry.aifoundry_helper import AIFoundryManager
from src.pipeline.promptEngineering.models import PatientInformation, PhysicianInformation, ClinicalInformation

load_dotenv()

patient_info = PatientInformation(
    patient_name="Sarah Sample",
    patient_date_of_birth="10/19/2014",
    patient_id="4567890",
    patient_address="25 W Randolph St, Chicago, IL, 60601",
    patient_phone_number="555-123-4567"
)

# Instantiate PhysicianInformation including a physician contact dictionary.
physician_info = PhysicianInformation(
    physician_name="Shiva Pedram, MD",
    specialty="Pediatric Gastroenterology",
    physician_contact={
        "office_phone": "555-991-2750",
        "fax": "555-786-5643",
        "office_address": "5721 S Maryland Ave, Chicago, IL 60637"
    }
)

# Instantiate ClinicalInformation with detailed clinical context.
clinical_info = ClinicalInformation(
    diagnosis="Crohn's Disease; Anemia 2/2 blood loss",
    icd_10_code="K50.90; D50.9",
    prior_treatments_and_results=(
        "Methylprednisolone 40mg daily; Patient continued to have bloody stools and abdominal pain with no improvement"
    ),
    specific_drugs_taken_and_failures=(
        "Methylprednisolone 40mg daily; Patient continued to have bloody stools and abdominal pain with no improvement"
    ),
    alternative_drugs_required="Not provided",
    relevant_lab_results_or_imaging=(
        "Hemoglobin: 9.0 g/dL; Hematocrit: 32%; Platelets: 450,000 cells/µL; WBC: 12,000 cells/µL; "
        "CRP: 25 mg/L; Fecal Calprotectin: 150 µg/g; additional lab/imaging details as needed."
    ),
    symptom_severity_and_impact=(
        "Patient has multiple episodes of abdominal cramping and bowel movements with visible hematochezia; "
        "Patient appears pale, tired but interactive; Positive for fatigue, pallor, abdominal pain, hematochezia, "
        "dizziness, blood loss (by rectum); Mild tachycardia present"
    ),
    prognosis_and_risk_if_not_approved=(
        "Patient continues to have frequent blood in stools and anemia as well as abdominal discomfort despite current steroid therapy"
    ),
    clinical_rationale_for_urgency=(
        "Urgent (In checking this box, I attest to the fact that applying the standard review time frame may seriously jeopardize "
        "the customer's life, health, or ability to regain maximum function)"
    ),
    treatment_request={
        "name_of_medication_or_procedure": "Adalimumab",
        "code_of_medication_or_procedure": "Not provided",
        "dosage": (
            "160 mg (given as four 40 mg injections on day 1) followed by 80 mg (given as two 40 mg injections) two weeks later. "
            "40mg every other week starting 2 weeks from 2nd dose"
        ),
        "duration": "6 months",
        "rationale": (
            "Patient will likely need to initiate biologic therapy given severity of symptoms despite current therapy"
        ),
        "presumed_eligibility": "Not provided"
    }
)

In [2]:
policy_text='''<figure>\n\ncigna\nhealthcare\n\n</figure>\n\n\n# PRIOR AUTHORIZATION POLICY\n\nPOLICY:\n\nInflammatory Conditions - Adalimumab Products Prior Authorization\n\nPolicy\n\n· Abrilada™ (adalimumab-afzb subcutaneous injection - Pfizer)\n\n· adalimumab-aacf subcutaneous injection (Fresenius Kabi)\n\n· adalimumab-adaz subcutaneous injection (Sandoz/Novartis)\n\n· adalimumab-adbm subcutaneous injection (Boehringer Ingelheim)\n\n· adalimumab-fkjp subcutaneous injection (Mylan)\n\n· adalimumab-ryvk subcutaneous injection (Teva/Alvotech)\n\n· Amjevita® (adalimumab-atto subcutaneous injection - Amgen)\n\n· Cyltezo® (adalimumab-adbm subcutaneous injection - Boehringer\nIngelheim)\n\n· Hadlima™ (adalimumab-bwwd\nsubcutaneous\ninjection\n–\n\nOrganon/Samsung Bioepis)\n\n· Hulio® (adalimumab-fkjp subcutaneous injection - Mylan)\n\n· Humira® (adalimumab subcutaneous injection - AbbVie, Cordavis)\n\n. Hyrimoz® (adalimumab-adaz\nsubcutaneous\nSandoz/Novartis, Cordavis)\ninjection\n–\n\n· Idacio® (adalimumab-aacf subcutaneous injection - Fresenius Kabi)\n\n· Simlandi® (adalimumab-ryvk subcutaneous injection -\nTeva/Alvotech)\n\n· Yuflyma® (adalimumab-aaty subcutaneous injection - Celltrion)\n\n· Yusimry™ (adalimuamb-aqvh subcutaneous injection - Coherus)\n\nREVIEW DATE: 03/27/2024; selected revision 04/03/2024, 09/11/2024\n\n\n## INSTRUCTIONS FOR USE\n\nTHE FOLLOWING COVERAGE POLICY APPLIES TO HEALTH BENEFIT PLANS ADMINISTERED BY CIGNA COMPANIES. CERTAIN CIGNA\nCOMPANIES AND/OR LINES OF BUSINESS ONLY PROVIDE UTILIZATION REVIEW SERVICES TO CLIENTS AND DO NOT MAKE COVERAGE\nDETERMINATIONS. REFERENCES TO STANDARD BENEFIT PLAN LANGUAGE AND COVERAGE DETERMINATIONS DO NOT APPLY TO THOSE\nCLIENTS. COVERAGE POLICIES ARE INTENDED TO PROVIDE GUIDANCE IN INTERPRETING CERTAIN STANDARD BENEFIT PLANS\nADMINISTERED BY CIGNA COMPANIES. PLEASE NOTE, THE TERMS OF A CUSTOMER\'S PARTICULAR BENEFIT PLAN DOCUMENT [GROUP\nSERVICE AGREEMENT, EVIDENCE OF COVERAGE, CERTIFICATE OF COVERAGE, SUMMARY PLAN DESCRIPTION (SPD) OR SIMILAR PLAN\nDOCUMENT] MAY DIFFER SIGNIFICANTLY FROM THE STANDARD BENEFIT PLANS UPON WHICH THESE COVERAGE POLICIES ARE BASED. FOR\nEXAMPLE, A CUSTOMER\'S BENEFIT PLAN DOCUMENT MAY CONTAIN A SPECIFIC EXCLUSION RELATED TO A TOPIC ADDRESSED IN A COVERAGE\nPOLICY. IN THE EVENT OF A CONFLICT, A CUSTOMER\'S BENEFIT PLAN DOCUMENT ALWAYS SUPERSEDES THE INFORMATION IN THE\nCOVERAGE POLICIES. IN THE ABSENCE OF A CONTROLLING FEDERAL OR STATE COVERAGE MANDATE, BENEFITS ARE ULTIMATELY\nDETERMINED BY THE TERMS OF THE APPLICABLE BENEFIT PLAN DOCUMENT. COVERAGE DETERMINATIONS IN EACH SPECIFIC INSTANCE\nREQUIRE CONSIDERATION OF 1) THE TERMS OF THE APPLICABLE BENEFIT PLAN DOCUMENT IN EFFECT ON THE DATE OF SERVICE; 2) ANY\nAPPLICABLE LAWS/REGULATIONS; 3) ANY RELEVANT COLLATERAL SOURCE MATERIALS INCLUDING COVERAGE POLICIES AND; 4) THE\nSPECIFIC FACTS OF THE PARTICULAR SITUATION. EACH COVERAGE REQUEST SHOULD BE REVIEWED ON ITS OWN MERITS. MEDICAL\nDIRECTORS ARE EXPECTED TO EXERCISE CLINICAL JUDGMENT AND HAVE DISCRETION IN MAKING INDIVIDUAL COVERAGE\nDETERMINATIONS. COVERAGE POLICIES RELATE EXCLUSIVELY TO THE ADMINISTRATION OF HEALTH BENEFIT PLANS. COVERAGE POLICIES\nARE NOT RECOMMENDATIONS FOR TREATMENT AND SHOULD NEVER BE USED AS TREATMENT GUIDELINES. IN CERTAIN MARKETS,\nDELEGATED VENDOR GUIDELINES MAY BE USED TO SUPPORT MEDICAL NECESSITY AND OTHER COVERAGE DETERMINATIONS.\n\n\n# CIGNA NATIONAL FORMULARY COVERAGE:\n\nPage 1 of 21 - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\n\n## OVERVIEW\n\nAdalimumab products are tumor necrosis factor inhibitors (TNFis) approved for the following\nuses: 1\n\n· Ankylosing spondylitis, for reducing signs and symptoms in patients with active\ndisease.\n\n· Crohn\'s disease, for treatment of moderately to severely active disease in patients\n≥ 6 years of age.\n\n· Hidradenitis suppurativa, for the treatment of moderate to severe disease in\npatients ≥ 12 years of age.\n\n· Juvenile idiopathic arthritis (JIA), ± methotrexate for reducing signs and\nsymptoms of moderately to severely active polyarticular disease in patients ≥ 2\nyears of age.\n\n· Plaque psoriasis, for the treatment of adults with moderate to severe chronic\ndisease who are candidates for systemic therapy or phototherapy and when other\nsystemic therapies are medically less appropriate.\n\n· Psoriatic arthritis (PsA), ± conventional synthetic disease-modifying\nantirheumatic drugs (DMARDs), for reducing the signs and symptoms of active\narthritis, inhibiting the progression of structural damage, and improving physical\nfunction.\n\n. Rheumatoid arthritis, + methotrexate or other conventional synthetic DMARDS to\nreduce the signs and symptoms, induce major clinical response, inhibit the\nprogression of structural damage, and improve physical function in adults with\nmoderately to severely active disease.\n\n· Ulcerative colitis, for treatment of moderately to severely active disease in patients\n≥ 5 years of age. However, efficacy has not been established in patients with\nulcerative colitis who have lost response or were intolerant to another TNFi.\n\n· Uveitis, in patients ≥ 2 years of age with noninfectious intermediate, posterior, and\npanuveitis.\n\n\n### Guidelines\n\nTNFis are featured prominently in guidelines for treatment of inflammatory conditions.\n\n· Ankylosing Spondylitis and Spondyloarthritis: Guidelines for ankylosing\nspondylitis and non-radiographic axial spondylitis are published by the American\nCollege of Rheumatology (ACR)/Spondylitis Association of America/Spondyloarthritis\nResearch and Treatment Network (2019).3 TNFis are recommended as the initial\nbiologic. In those who are secondary non-responders to a TNFi, a second TNFi is\nrecommended over switching out of the class.\n\n· Crohn\'s Disease: The American College of Gastroenterology (ACG) has guidelines\nfor Crohn\'s disease (2018).4 TNFis are listed as an option for disease that is resistant\nto corticosteroids, severely active disease, perianal fistulizing disease, and\nmaintenance of remission. In post-operative Crohn\'s disease, a TNFi should be\nstarted within 4 weeks of surgery to prevent recurrence. Guidelines from the\nAmerican Gastroenterological Association (AGA) [2021] include TNFis among the\ntherapies for moderate to severe Crohn\'s disease, for induction and maintenance of\nremission. 17\n\n· Hidradenitis Suppurativa: British guidelines (2018) recommend consideration of\nadalimumab for those with moderate to severe disease who do not respond to\nconventional therapy.19\n\n· JIA: There are guidelines from ACR and the Arthritis Foundation for the treatment\nof JIA (2021) which address oligoarthritis and temporomandibular joint (TMJ)\narthritis. For oligoarthritis, a biologic is recommended following a trial of a\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\nconventional synthetic DMARD.6 In patients with TMJ arthritis, scheduled\nnonsteroidal anti-inflammatory drugs (NSAIDs) and/or intra-articular glucocorticoids\nare recommended first-line. A biologic is a therapeutic option if there is an\ninadequate response or intolerance. Additionally, rapid escalation to a biologic ±\nconventional synthetic DMARD (methotrexate preferred) is often appropriate given\nthe impact and destructive nature of TMJ arthritis. In these guidelines, there is not a\npreferred biologic that should be initiated for JIA. ACR guidelines (2019) are also\navailable specifically for juvenile non-systemic polyarthritis, sacroiliitis, and\nenthesitis.5 TNFis are the biologics recommended for polyarthritis, sacroiliitis, and\nenthesitis. Biologics are recommended following other therapies (e.g., following\nDMARDs for active polyarthritis or following an NSAID for active JIA with sacroiliitis\nor enthesitis). However, there are situations where initial therapy with a biologic may\nbe preferred over other conventional therapies (e.g., if there is involvement of high-\nrisk joints such as the cervical spine, wrist, or hip; high disease activity; and/or\nthose judged to be at high risk of disabling joint damage).\n\n· Plaque Psoriasis: Guidelines from the American Academy of Dermatologists and\nNational Psoriasis Foundation (2019) recommend adalimumab as a monotherapy\ntreatment option for adults with moderate to severe disease.7\n\n· PsA: Guidelines from ACR (2019) recommend TNFis over other biologics for use in\ntreatment-naïve patients with PsA and in those who were previously treated with an\noral therapy.8\n\n· Rheumatoid Arthritis: Guidelines from ACR (2021) recommend addition of a\nbiologic or a targeted synthetic DMARD for a patient taking the maximum tolerated\ndose of methotrexate who is not at target.2\n\n. Ulcerative Colitis: Guidelines from the ACG for ulcerative colitis (2019) note that\nthe following agents can be used for induction of remission in moderately to severely\nactive disease: budesonide extended-release tablets, oral or intravenous systemic\ncorticosteroids, Entyvio® (vedolizumab intravenous infusion), Xeljanz®/XR (tofacitinib\ntablets/extended-release tablets), or TNFis.9 Guidelines from the AGA (2020)\nrecommend Xeljanz only after failure of or intolerance to a TNFi.10 In addition to the\napproved indication, clinical guidelines for the management of pouchitis (2009)\nindicate that first-line therapy for pouchitis is antibiotic therapy (e.g. metronidazole,\nciprofloxacin).11 Other treatment options include maintenance probiotics, oral or\ntopical budesonide, anti-inflammatory drugs (e.g., mesalamine), or\nimmunosuppressive agents (e.g., infliximab).\n\n. Uveitis and Ocular Inflammatory Disorders: American Academy of\nOphthalmology (AAO) guidelines (2014) note that adalimumab may be used in\npatients with uveitis due to various causes (e.g., spondyloarthropathy-associated or\nhuman leukocyte antigen [HLA]-B27-associated uveitis, JIA-associated uveitis, and\nother posterior uveitides and panuveitis syndromes).12 Adalimumab should be\nconsidered second-line in vision-threatening JIA-associated uveitis when\nmethotrexate has failed or is not tolerated (strong recommendation) and may be\nused as corticosteroid-sparing treatment for vision-threatening chronic uveitis from\nseronegative spondyloarthropathy (strong recommendation). Adalimumab may also\nbe considered in other patients who have vision-threatening or corticosteroid-\ndependent disease who have failed first-line therapies. Adalimumab should be\nconsidered as a second-line immunomodulatory agent for severe ocular inflammatory\nconditions including chronic and severe scleritis. ACR/Arthritis Federation guidelines\n(2019) for uveitis associated with JIA make recommendations for use of conventional\nsystemic DMARDs and biologics. In patients with severe active chronic anterior\nuveitis associated with sight-threatening complications, a TNFi (monoclonal antibody)\n\\+ methotrexate is recommended.19\n\n<!-- PageFooter="3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior Authorization Policy" -->\n<!-- PageBreak -->\n\n\n## Other Uses with Supportive Evidence\n\nThere are guidelines and/or published data supporting the use of adalimumab products in\nthe following conditions:\n\n. Behcet\'s Disease: The European Union Against Rheumatism (EULAR)\nrecommendations (2018) include TNFis for initial or recurrent sight-threatening\nuveitis.13 For patients refractory to first-line treatments (e.g., corticosteroids), TNFis\nare among the treatment options for mucocutaneous manifestations, venous\nthrombosis, severe or refractory gastrointestinal disease, and recurrent/chronic joint\ninvolvement. Recommendations for the use of TNFis in ocular inflammatory\ndisorders from the AAO (2014) note that TNFis may be used first-line in patients with\nophthalmic manifestations of Behcet\'s disease and for acute exacerbations of\npreexisting Behcet\'s disease. 12\n\n· Pyoderma Gangrenosum: Although guidelines are not current, multiple topical\nand systemic therapies have been used for pyoderma gangrenosum. Oral prednisone\nis the most common initial immunosuppressant medication. 14 Other systemic\ntherapies include cyclosporine, methotrexate, azathioprine, cyclophosphamide,\nmycophenolate mofetil, and TNFis (i.e., infliximab, etanercept, and adalimumab\nproducts). In case reports, TNFis have been effective.\n\n· Sarcoidosis: According to European Respiratory Society guidelines for sarcoidosis\n(2021), a TNFi is recommended after a trial of glucocorticoids and\nimmunosuppressants for pulmonary and neurosarcoidosis.15\n\n\n## POLICY STATEMENT\n\nPrior Authorization is recommended for prescription benefit coverage of adalimumab\nproducts. All approvals are provided for the duration noted below. In cases where the\napproval is authorized in months, 1 month is equal to 30 days. Because of the specialized\nskills required for evaluation and diagnosis of patients treated with adalimumab products as\nwell as the monitoring required for adverse events and long-term efficacy, initial approval\nrequires the agent to be prescribed by or in consultation with a physician who specializes in\nthe condition being treated.\n\n· Abrilada™ (adalimumab-afzb subcutaneous injection - Pfizer)\n\n· adalimumab-aacf subcutaneous injection (Fresenius Kabi)\n\n· adalimumab-adaz subcutaneous injection (Sandoz/Novartis)\n\n· adalimumab-adbm subcutaneous injection (Boehringer Ingelheim)\n\n· adalimumab-fkjp subcutaneous injection (Mylan)\n\n· adalimumab-ryvk subcutaneous injection (Teva/Alvotech)\n\n· Amjevita (adalimumab-atto subcutaneous injection - Amgen)\n\n· Cyltezo® (adalimumab-adbm subcutaneous injection - Boehringer\nIngelheim)\n\n· Hadlima™ (adalimumab-bwwd subcutaneous injection - Organon/Samsung\nBioepis)\n\n. Hulio® (adalimumab-fkjp subcutaneous injection - Mylan)\n\n· Humira® (adalimumab subcutaneous injection - AbbVie, Cordavis)\n\n· Hyrimoz® (adalimumab-adaz subcutaneous injection - Sandoz/Novartis,\nCordavis)\n\n· Idacio® (adalimumab-aacf subcutaneous injection - Fresenius Kabi)\n\n· Simlandi® (adalimumab-ryvk subcutaneous injection - Teva/Alvotech)\n\n· Yuflyma® (adalimumab-aaty subcutaneous injection - Celltrion)\n\n· Yusimry™ (adalimuamb-aqvh subcutaneous injection - Coherus)\n\nis(are) covered as medically necessary when the following criteria is(are) met for\nFDA-approved indication(s) or other uses with supportive evidence (if applicable):\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\n\n# FDA-Approved Indications\n\n1\\. Ankylosing Spondylitis. Approve for the duration noted if the patient meets ONE of\nthe following (A or B):\n\nA) Initial Therapy. Approve for 6 months if the patient meets BOTH of the following (i\nand ii):\n\ni. Patient is ≥ 18 years of age; AND\n\nii. The medication is prescribed by or in consultation with a rheumatologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets BOTH of the following (i and ii):\n\ni. Patient has been established on therapy for at least 6 months; AND\nNote: A patient who has received < 6 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\nii. Patient meets at least ONE of the following (a or b):\n\na) When assessed by at least one objective measure, patient experienced a\nbeneficial clinical response from baseline (prior to initiating an adalimumab\nproduct); OR\nNote: Examples of objective measures include Ankylosing Spondylitis Disease\nActivity Score (ASDAS), Ankylosing Spondylitis Quality of Life Scale (ASQoL),\nBath Ankylosing Spondylitis Disease Activity Index (BASDAI), Bath Ankylosing\nSpondylitis Functional Index (BASFI), Bath Ankylosing Spondylitis Global\nScore (BAS-G), Bath Ankylosing Spondylitis Metrology Index (BASMI),\nDougados Functional Index (DFI), Health Assessment Questionnaire for the\nSpondyloarthropathies (HAQ-S), and/or serum markers (e.g., C-reactive\nprotein, erythrocyte sedimentation rate).\n\nb) Compared with baseline (prior to initiating an adalimumab product), patient\nexperienced an improvement in at least one symptom, such as decreased\npain or stiffness, or improvement in function or activities of daily living.\n\n2\\. Crohn\'s Disease. Approve for the duration noted if the patient meets ONE of the\nfollowing (A or B):\n\nA) Initial Therapy. Approve for 6 months if the patient meets ALL of the following (i, ii,\nand iii):\n\ni. Patient is ≥ 6 years of age; AND\n\nii. Patient meets ONE of the following (a, b, c, or d):\n\na) Patient has tried or is currently taking corticosteroids, or corticosteroids are\ncontraindicated in this patient; OR\nNote: Examples of corticosteroids are prednisone or methylprednisolone.\n\nb) Patient has tried one other conventional systemic therapy for Crohn\'s disease;\nOR\nNote: Examples of conventional systemic therapy for Crohn\'s disease include\nazathioprine, 6-mercaptopurine, or methotrexate. An exception to the\nrequirement for a trial of or contraindication to steroids or a trial of one other\nconventional systemic agent can be made if the patient has already tried at\nleast one biologic other than the requested medication. A biosimilar of the\nrequested biologic does not count. Refer to Appendix for examples of biologics\nused for Crohn\'s disease. A trial of mesalamine does not count as a systemic\nagent for Crohn\'s disease.\n\nc) Patient has enterocutaneous (perianal or abdominal) or rectovaginal fistulas;\nOR\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\nd) Patient had ileocolonic resection (to reduce the chance of Crohn\'s disease\nrecurrence); AND\n\niii. The medication is prescribed by or in consultation with a gastroenterologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets BOTH of the following (i and ii):\n\ni. Patient has been established on therapy for at least 6 months; AND\nNote: A patient who has received < 6 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\nii. Patient meets at least ONE of the following (a or b):\n\na) When assessed by at least one objective measure, patient experienced a\nbeneficial clinical response from baseline (prior to initiating an adalimumab\nproduct); OR\n\nNote: Examples of objective measures include fecal markers (e.g., fecal\nlactoferrin, fecal calprotectin), serum markers (e.g., C-reactive protein),\nimaging studies (magnetic resonance enterography [MRE], computed\ntomography enterography [CTE]), endoscopic assessment, and/or reduced\ndose of corticosteroids.\n\nb) Compared with baseline (prior to initiating an adalimumab product), patient\nexperienced an improvement in at least one symptom, such as decreased\npain, fatigue, stool frequency, and/or blood in stool.\n\n3\\. Juvenile Idiopathic Arthritis (JIA) [or juvenile rheumatoid arthritis]\n{regardless of type of onset}. Approve for the duration noted if the patient meets\nONE of the following (A or B):\n\nNote: This includes a patient with juvenile spondyloarthropathy/active sacroiliac\narthritis.\n\nA) Initial Therapy. Approve for 6 months if the patient meets ALL of the following (i, ii,\nand iii):\n\ni. Patient is ≥ 2 years of age; AND\n\nii. Patient meets ONE of the following (a, b, c, or d):\na) Patient has tried one other systemic therapy for this condition; OR\nNote: Examples of other systemic therapies for JIA include methotrexate,\nsulfasalazine, leflunomide, or a nonsteroidal anti-inflammatory drug (NSAID)\n[e.g., ibuprofen, naproxen]. A previous trial of one biologic other than the\nrequested medication also counts as a trial of one systemic therapy for JIA. A\nbiosimilar of the requested biologic does not count. Refer to Appendix for\nexamples of biologics used for JIA.\n\nb) Patient will be starting on adalimumab concurrently with methotrexate,\nsulfasalazine, or leflunomide; OR\n\nc) Patient has an absolute contraindication to methotrexate, sulfasalazine, or\nleflunomide; OR\nNote: Examples of contraindications to methotrexate include pregnancy,\nbreast feeding, alcoholic liver disease, immunodeficiency syndrome, blood\ndyscrasias.\n\nd) Patient has aggressive disease, as determined by the prescriber; AND\niii. The medication is prescribed by or in consultation with a rheumatologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets BOTH of the following (i and ii):\n\ni. Patient has been established on therapy for at least 6 months; AND\nNote: A patient who has received < 6 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\nii. Patient meets at least ONE of the following (a or b):\n\na) When assessed by at least one objective measure, patient experienced a\nbeneficial clinical response from baseline (prior to initiating an adalimumab\nproduct); OR\n\nNote: Examples of objective measures include Physician Global Assessment\n(MD global), Parent/Patient Global Assessment of Overall Well-Being (PGA),\nParent/Patient Global Assessment of Disease Activity (PDA), Juvenile Arthritis\nDisease Activity Score (JDAS), Clinical Juvenile Arthritis Disease Activity Score\n(cJDAS), Juvenile Spondyloarthritis Disease Activity Index (JSpADA), serum\nmarkers (e.g., C-reactive protein, erythrocyte sedimentation rate), and/or\nreduced dosage of corticosteroids.\n\nb) Compared with baseline (prior to initiating an adalimumab product), patient\nexperienced an improvement in at least one symptom, such as improvement\nin limitation of motion, less joint pain or tenderness, decreased duration of\nmorning stiffness or fatigue, or improved function or activities of daily living.\n\n4\\. Hidradenitis Suppurativa. Approve for the duration noted if the patient meets ONE of\nthe following (A or B):\n\nA) Initial Therapy. Approve for 3 months if the patients meets ALL of the following (i, ii,\nand iii):\n\ni. Patient is ≥ 12 years of age; AND\n\nii. Patient has tried at least ONE other therapy; AND\n\nNote: Examples include intralesional or oral corticosteroids (such as\ntriamcinolone or prednisone), systemic antibiotics (e.g., clindamycin, dicloxacillin,\nor erythromycin), or isotretinoin.\n\niii. The medication is prescribed by or in consultation with a dermatologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets ALL of the following (i, ii, and iii):\n\ni. Patient has been established on therapy for at least 3 months; AND\nNote: A patient who has received < 3 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\nii. When assessed by at least one objective measure, patient experienced a\nbeneficial clinical response from baseline (prior to initiating an adalimumab\nproduct); AND\n\nNote: Examples of objective measures include Hurley staging, Sartorius score,\nPhysician Global Assessment, and Hidradenitis Suppurativa Severity Index.\n\niii. Compared with baseline (prior to initiating an adalimumab product), patient\nexperienced an improvement in at least one symptom, such as decreased pain or\ndrainage of lesions, nodules, or cysts.\n\n5\\. Plaque Psoriasis. Approve for the duration noted if the patient meets ONE of the\nfollowing (A or B):\n\nA) Initial Therapy. Approve for 3 months if the patient meets ALL of the following (i, ii,\n\nand iii):\n\ni. Patient is ≥ 18 years of age; AND\n\nii. Patient meets ONE of the following (a or b):\n\na) Patient has tried at least one traditional systemic agent for psoriasis for at\nleast 3 months, unless intolerant; OR\n\nNote: Examples include methotrexate, cyclosporine, or acitretin. A 3-month\ntrial of psoralen plus ultraviolet A light (PUVA) also counts. An exception to\nthe requirement for a trial of one traditional systemic agent for psoriasis can\nbe made if the patient has already had a 3-month trial or previous intolerance\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\nto at least one biologic other than the requested medication. A biosimilar of\nthe requested biologic does not count. Refer to Appendix for examples of\nbiologics used for psoriasis. A patient who has already tried a biologic for\npsoriasis is not required to "step back" and try a traditional systemic agent for\npsoriasis.\n\nb) Patient has a contraindication to methotrexate, as determined by the\nprescriber; AND\n\niii. The medication is prescribed by or in consultation with a dermatologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets ALL of the following (i, ii, and iii):\n\ni. Patient has been established on therapy for at least 3 months; AND\nNote: A patient who has received < 3 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\nii. Patient experienced a beneficial clinical response, defined as improvement from\nbaseline (prior to initiating an adalimumab product) in at least one of the\nfollowing: estimated body surface area affected, erythema, induration/thickness,\nand/or scale of areas affected by psoriasis; AND\n\niii. Compared with baseline (prior to receiving an adalimumab product), patient\nexperienced an improvement in at least one symptom, such as decreased pain,\nitching, and/or burning.\n\n6\\. Psoriatic Arthritis. Approve for the duration noted if the patient meets ONE of the\nfollowing (A or B):\n\nA) Initial Therapy. Approve for 6 months if the patient meets BOTH of the following (i\nand ii):\n\ni. Patient is ≥ 18 years of age; AND\n\nii. The medication is prescribed by or in consultation with a rheumatologist or a\ndermatologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets BOTH of the following (i and ii):\n\ni. Patient has been established on therapy for at least 6 months; AND\nNote: A patient who has received < 6 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\nii. Patient meets at least ONE of the following (a or b):\n\na) When assessed by at least one objective measure, patient experienced a\nbeneficial clinical response from baseline (prior to initiating an adalimumab\nproduct); OR\nNote: Examples of objective measures of disease activity include Disease\nActivity Index for Psoriatic Arthritis (DAPSA), Composite Psoriatic Disease\nActivity Index (CPDAI), Psoriatic Arthritis Disease Activity Score (PsA DAS),\nGrace Index, Leeds Enthesitis Score (LEI), Spondyloarthritis Consortium of\nCanada (SPARCC) enthesitis score, Leeds Dactylitis Instrument Score,\nMinimal Disease Activity (MDA), Psoriatic Arthritis Impact of Disease (PsAID-\n12), and/or serum markers (e.g., C-reactive protein, erythrocyte\nsedimentation rate).\n\nb) Compared with baseline (prior to initiating an adalimumab product), patient\nexperienced an improvement in at least one symptom, such as less joint pain,\nmorning stiffness, or fatigue; improved function or activities of daily living; or\ndecreased soft tissue swelling in joints or tendon sheaths.\n\n<!-- PageFooter="3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior Authorization Policy" -->\n<!-- PageBreak -->\n\n7\\. Rheumatoid Arthritis. Approve for the duration noted if the patient meets ONE of the\nfollowing (A or B):\n\nA) Initial Therapy. Approve for 6 months if the patient meets ALL of the following (i, ii,\nand iii):\n\ni. Patient is ≥ 18 years of age; AND\n\nii. Patient has tried ONE conventional synthetic disease-modifying antirheumatic\ndrug (DMARD) for at least 3 months; AND\n\nNote: Examples include methotrexate (oral or injectable), leflunomide,\nhydroxychloroquine, and sulfasalazine. An exception to the requirement for a trial\nof one conventional synthetic DMARD can be made if the patient has already had\na 3-month trial with at least one biologic other than the requested medication. A\nbiosimilar of the requested biologic does not count. Refer to Appendix for\nexamples of biologics used for rheumatoid arthritis. A patient who has already\ntried a biologic for rheumatoid arthritis is not required to "step back" and try a\nconventional synthetic DMARD.\n\nii. The medication is prescribed by or in consultation with a rheumatologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets BOTH of the following (i and ii):\n\ni. Patient has been established on therapy for at least 6 months; AND\nNote: A patient who has received < 6 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\nii. Patient meets at least ONE of the following (a or b):\n\na) Patient experienced a beneficial clinical response when assessed by at least\none objective measure; OR\nNote: Examples of objective measures of disease activity include Clinical\nDisease Activity Index (CDAI), Disease Activity Score (DAS) 28 using\nerythrocyte sedimentation rate or C-reactive protein, Patient Activity Scale\n(PAS)-II, Rapid Assessment of Patient Index Data 3 (RAPID-3), and/or\nSimplified Disease Activity Index (SDAI).\n\nb) Patient experienced an improvement in at least one symptom, such as\ndecreased joint pain, morning stiffness, or fatigue; improved function or\nactivities of daily living; or decreased soft tissue swelling in joints or tendon\nsheaths.\n\n8\\. Ulcerative Colitis. Approve for the duration noted if the patient meets ONE of the\nfollowing (A or B):\n\nA) Initial Therapy. Approve for 6 months if the patient meets ALL of the following (i, ii,\nand iii):\n\ni. Patient is ≥ 5 years of age; AND\n\nii. Patient meets ONE of the following (a or b):\na) Patient has tried one systemic therapy; OR\n\nNote: Examples include 6-mercaptopurine, azathioprine, cyclosporine,\ntacrolimus, or a corticosteroid such as prednisone or methylprednisolone. A\ntrial of one biologic other than the requested medication also counts as a trial\nof one systemic agent for ulcerative colitis. A biosimilar of the requested\nbiologic does not count. Refer to Appendix for examples of biologics used for\nulcerative colitis.\n\nb) Patient meets BOTH of the following [(1) and (2)]:\n\n(1)Patient has pouchitis; AND\n(2)Patient has tried an antibiotic, probiotic, corticosteroid enema, or\nmesalamine enema; AND\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\nNote: Examples of antibiotics include metronidazole and ciprofloxacin.\nExamples of corticosteroid enemas include hydrocortisone enema.\n\niii. The medication is prescribed by or in consultation with a gastroenterologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets BOTH of the following (i and ii):\n\ni. Patient has been established on therapy for at least 6 months; AND\nNote: A patient who has received < 6 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\nii. Patient meets at least ONE of the following (a or b):\n\na) When assessed by at least one objective measure, patient experienced a\nbeneficial clinical response from baseline (prior to initiating an adalimumab\nproduct); OR\nNote: Examples of objective measures include fecal markers (e.g., fecal\ncalprotectin), serum markers (e.g., C-reactive protein), endoscopic\nassessment, and/or reduced dose of corticosteroids.\n\nb) Compared with baseline (prior to initiating an adalimumab product), patient\nexperienced an improvement in at least one symptom, such as decreased\npain, fatigue, stool frequency, and/or rectal bleeding.\n\n9\\. Uveitis (including other posterior uveitides and panuveitis syndromes). Approve\nfor the duration noted if the patient meets ONE of the following (A or B):\n\nA) Initial Therapy. Approve for 6 months if the patient meets ALL of the following (i, ii,\nand iii):\n\ni. Patient is ≥ 2 years of age; AND\n\nii. Patient has tried ONE of the following therapies: periocular, intraocular, or\nsystemic corticosteroids; immunosuppressives; AND\nNote: Examples of corticosteroids include prednisolone, triamcinolone,\nbetamethasone, methylprednisolone, and prednisone. Examples of\nimmunosuppressive agents include methotrexate, mycophenolate mofetil,\nazathioprine, and cyclosporine. A trial of one biologic other than the requested\nmedication also counts. A biosimilar of the requested biologic does not count.\n\niii. The medication is prescribed by or in consultation with an ophthalmologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets BOTH of the following (i and ii):\n\ni. Patient has been established on therapy for at least 6 months; AND\nNote: A patient who has received < 6 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\nii. Patient meets at least one of the following (a or b):\n\na) When assessed by at least one objective measure, patient experienced a\nbeneficial clinical response from baseline (prior to initiating an adalimumab\nproduct); OR\n\nNote: Examples of objective measures include best-corrected visual acuity,\nassessment of chorioretinal and/or inflammatory retinal vascular lesions, or\nanterior chamber cell grade or vitreous haze grade.\n\nb) Compared with baseline (prior to initiating an adalimumab product), patient\nexperienced an improvement in at least one symptom, such as decreased eye\npain, redness, light sensitivity, and/or blurred vision; or improvement in\nvisual acuity.\n\n\n# Other Uses with Supportive Evidence\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\n10\\. Behcet\'s Disease. Approve for the duration noted if the patient meets ONE of the\nfollowing (A or B):\n\nA) Initial Therapy. Approve for 3 months if the patient meets ALL of the following (i, ii,\nand ii):\n\ni. Patient is > 2 years of age; AND\n\nii. Patient meets ONE of the following (a or b):\na) Patient has tried at least ONE conventional therapy; OR\nNote: Examples include systemic corticosteroids (e.g., methylprednisolone),\nimmunosuppressants (e.g., azathioprine, methotrexate, mycophenolate\nmofetil, cyclosporine, tacrolimus, Leukeran [chlorambucil tablets],\ncyclophosphamide, interferon alfa). A trial of one biologic other than the\nrequested medication also counts. A patient who has already tried one\nbiologic other than the requested drug for Behcet\'s disease is not required to\n"step back" and try a conventional therapy. A biosimilar of the requested\nbiologic does not count.\n\nb) Patient has ophthalmic manifestations of Behcet\'s disease; AND\n\niii. The medication is prescribed by or in consultation with a rheumatologist,\ndermatologist, ophthalmologist, gastroenterologist, or neurologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets ALL of the following (i, ii, and iii):\n\ni. Patient has been established on therapy for at least 3 months; AND\nNote: A patient who has received < 3 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\nii. When assessed by at least one objective measure, patient experienced a\nbeneficial clinical response from baseline (prior to initiating an adalimumab\nproduct); AND\n\nNote: Examples of objective measures are dependent upon organ involvement\nbut may include best-corrected visual acuity (if ophthalmic manifestations);\nserum markers (e.g., C-reactive protein, erythrocyte sedimentation rate); or\nulcer depth, number, and/or lesion size.\n\niii. Compared with baseline (prior to initiating an adalimumab product), patient\nexperienced an improvement in at least one symptom, such as decreased pain or\nimproved visual acuity (if ophthalmic manifestations).\n\n11\\. Pyoderma Gangrenosum. Approve for the duration noted if the patient meets ONE of\nthe following (A or B):\n\nA) Initial Therapy. Approve for 4 months if the patient meets ALL of the following (i, ii,\nand iii):\n\ni. Patient is > 18 years of age; AND\n\nii. Patient meets ONE of the following (a or b):\n\na) Patient has tried one systemic corticosteroid; OR\nNote: An example is prednisone.\n\nb) Patient has tried one other immunosuppressant for at least 2 months or was\nintolerant to one of these agents; AND\nNote: Examples include mycophenolate mofetil and cyclosporine.\n\niii. The medication is prescribed by or in consultation with a dermatologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets ALL of the following (i, ii, and iii):\n\ni. Patient has been established on therapy for at least 4 months; AND\nNote: A patient who has received < 4 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\nii. Patient experienced a beneficial clinical response, defined as improvement from\nbaseline (prior to initiating an adalimumab product) in at least one of the\nfollowing: size, depth, and/or number of lesions; AND\n\niii. Compared with baseline (prior to initiating an adalimumab product), patient\nexperienced an improvement in at least one symptom, such as decreased pain\nand/or tenderness of affected lesions.\n\n12\\. Sarcoidosis. Approve for the duration noted if the patient meets ONE of the following\n(A or B):\n\nA) Initial Therapy. Approve for 3 months if the patient meets ALL of the following (i, ii,\niii, and iv):\n\ni. Patient is > 18 years of age; AND\n\nii. Patient has tried at least one corticosteroid; AND\nNote: An example is prednisone.\n\niii. Patient has tried at least one immunosuppressive medication; AND\nNote: Examples include methotrexate, leflunomide, azathioprine, mycophenolate\nmofetil, cyclosporine, Leukeran (chlorambucil tablets), cyclophosphamide,\nThalomid (thalidomide capsules), an infliximab product, or chloroquine.\n\niv. The medication is prescribed by or in consultation with a pulmonologist,\nophthalmologist, or dermatologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets ALL of the following (i, ii, and iii):\n\ni. Patient has been established on therapy for at least 3 months; AND\nNote: A patient who has received < 3 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\nii. When assessed by at least one objective measure, patient experienced a\nbeneficial clinical response from baseline (prior to initiating an adalimumab\nproduct); AND\n\nNote: Examples of objective measures are dependent upon organ involvement\nbut may include lung function (e.g., predicted forced vital capacity and/or 6-\nminute walk distance); serum markers (e.g., C-reactive protein, liver enzymes,\nN-terminal pro-brain natriuretic peptide [NT-proBNP]); improvement in rash or\nskin manifestations, neurologic symptoms, or rhythm control; or imaging (e.g., if\nindicated, chest radiograph, magnetic resonance imaging [MRI], or\nechocardiography).\n\niii. Compared with baseline (prior to initiating an adalimumab product), patient\nexperienced an improvement in at least one symptom, such as decreased cough,\nfatigue, pain, palpitations, neurologic symptoms, and/or shortness of breath.\n\n13\\. Scleritis or Sterile Corneal Ulceration. Approve for the duration noted if the patient\nmeets ONE of the following (A or B):\n\nA) Initial Therapy. Approve for 6 months if the patient meets ALL of the following (i, ii,\nand iii):\n\ni. Patient is > 18 years of age; AND\n\nii. Patient has tried one other therapy for this condition; AND\n\nNote: Examples include oral nonsteroidal anti-inflammatory drugs (NSAIDs) such\nas indomethacin, naproxen, or ibuprofen; oral, topical (ophthalmic), or\nintravenous corticosteroids (such as prednisone, prednisolone,\nmethylprednisolone); methotrexate; cyclosporine; or other immunosupressants.\niii. The medication is prescribed by or in consultation with an ophthalmologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets BOTH of the following (i and ii):\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\ni. Patient has been established on therapy for at least 6 months; AND\n\nNote: A patient who has received < 6 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\nii. Patient meets at least ONE of the following (a or b):\n\na) When assessed by at least one objective measure, patient experienced a\nbeneficial clinical response from baseline (prior to initiating an adalimumab\nproduct); OR\n\nNote: Examples of objective measures are serum markers (e.g., C-reactive\nprotein, erythrocyte sedimentation rate).\n\nb) Compared with baseline (prior to initiating an adalimumab product), patient\nexperienced an improvement in at least one symptom, such as decreased eye\npain, redness, light sensitivity, tearing, and/or improvement in visual acuity.\n\n14\\. Spondyloarthritis, Other Subtypes. Approve for the duration noted if the patient\nmeets ONE of the following (A or B):\n\nNote: This includes undifferentiated arthritis, non-radiographic axial spondyloarthritis,\nreactive arthritis (Reiter\'s disease), or arthritis associated with inflammatory bowel\ndisease. For ankylosing spondylitis or psoriatic arthritis, refer to the respective criteria\nunder FDA-approved indications.\n\nA) Initial Therapy. Approve for 6 months if the patient meets ALL of the following (i, ii,\nand iii):\n\ni. Patient is > 18 years of age; AND\n\nii. Patient meets ONE of the following (a or b):\n\na) Patient meets BOTH of the following [(1) and (2)]:\n\n(1)Patient has arthritis primarily in the knees, ankles, elbows, wrists, hands,\nand/or feet; AND\n\n(2)Patient has tried at least one conventional synthetic disease-modifying\nantirheumatic drug (DMARD); OR\n\nNote: Examples include methotrexate, leflunomide, or sulfasalazine.\n\nb) Patient has axial spondyloarthritis AND has objective signs of inflammation,\ndefined as at least ONE of the following [(1) or (2)]:\n\n(1)C-reactive protein elevated beyond the upper limit of normal for the\nreporting laboratory; OR\n\n(2)Sacroiliitis reported on magnetic resonance imaging (MRI); AND\n\niii. The medication is prescribed by or in consultation with a rheumatologist.\n\nB) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the\npatient meets BOTH of the following (i and ii):\n\ni. Patient has been established on therapy for at least 6 months; AND\nNote: A patient who has received < 6 months of therapy or who is restarting\ntherapy with an adalimumab product is reviewed under criterion A (Initial\nTherapy).\n\nii. Patient meets at least ONE of the following (a or b):\n\na) When assessed by at least one objective measure, patient experienced a\nbeneficial clinical response from baseline (prior to initiating an adalimumab\nproduct); OR\n\nNote: Examples of objective measures include Ankylosing Spondylitis Disease\nActivity Score (ASDAS) and/or serum markers (e.g., C-reactive protein,\nerythrocyte sedimentation rate).\n\nb) Compared with baseline (prior to initiating an adalimumab product),\npatient experienced an improvement in at least one symptom, s\n\nc) Abrilada™ (adalimumab-afzb subcutaneous injection - Pfizer)\n\nd) adalimumab-aacf subcutaneous injection (Fresenius Kabi)\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\ne) adalimumab-adaz subcutaneous injection (Sandoz/Novartis)\n\nf) adalimumab-adbm subcutaneous injection (Boehringer Ingelheim)\n\ng) adalimumab-fkjp subcutaneous injection (Mylan)\n\nh) adalimumab-ryvk subcutaneous injection (Teva/Alvotech)\n\ni) Amjevita (adalimumab-atto subcutaneous injection - Amgen)\n\nj) Cyltezo® (adalimumab-adbm subcutaneous injection - Boehringer Ingelheim)\n\nk) Hadlima™ (adalimumab-bwwd subcutaneous injection - Organon/Samsung\nBioepis)\n\n1\\) Hulio® (adalimumab-fkjp subcutaneous injection - Mylan)\n\nm) Humira® (adalimumab subcutaneous injection - AbbVie, Cordavis)\n\nn) Hyrimoz® (adalimumab-adaz subcutaneous injection - Sandoz/Novartis,\nCordavis)\n\no) Idacio® (adalimumab-aacf subcutaneous injection - Fresenius Kabi)\n\np) Simlandi® (adalimumab-ryvk subcutaneous injection - Teva/Alvotech)\n\nq) Yuflyma® (adalimumab-aaty subcutaneous injection - Celltrion)\n\nr) Yusimry™ (adalimuamb-aqvh subcutaneous injection - Coherus)\n\ns) is(are) considered experimental, investigational, or unproven for ANY other\nuse(s) including the following (this list may not be all inclusive; criteria will be\nupdated as new published data are available):\n\nt) uch as decreased pain or stiffness, or improvement in function or activities of\ndaily living.\n\n\n# CONDITIONS NOT COVERED\n\n1\\. Concurrent Use with a Biologic or with a Targeted Synthetic Oral Small\nMolecule Drug. This medication should not be administered in combination with\nanother biologic or with a targeted synthetic oral small molecule drug used for an\ninflammatory condition (see Appendix for examples). Combination therapy is generally\nnot recommended due to a potentially higher rate of adverse events and lack of\ncontrolled clinical data supporting additive efficacy.\n\nNote: This does NOT exclude the use of conventional synthetic DMARDs (e.g.,\nmethotrexate, leflunomide, hydroxychloroquine, or sulfasalazine) in combination with\nthis medication.\n\n2\\. Polymyalgia Rheumatica (PMR). EULAR/ACR guidelines for the management of PMR\n(2015) strongly recommend against the use of TNFis for treatment of PMR.17 This\nrecommendation is based on lack of evidence for benefit as well as considerable\npotential for harm.\n\nREFERENCES\n\n1\\. Humira® subcutaneous injection [prescribing information]. North Chicago, IL: AbbVie;\nOctober 2023.\n\n2\\. Singh JA, Saag KG, Bridges SL Jr, et al. 2015 American College of Rheumatology\nGuideline for the treatment of rheumatoid arthritis. Arthritis Rheumatol. 2016;68(1):1-\n26\\.\n\n3\\. Ward MM, Deodhar A, Gensler LS, et al. 2019 update of the American College of\nRheumatology/Spondylitis Association of America/Spondyloarthritis Research and\nTreatment Network recommendations for the treatment of ankylosing spondylitis and\nnonradiographic axial spondyloarthritis. Arthritis Rheumatol. 2019;71(10):1599-1613.\n\n4\\. Lichtenstein GR, Loftus EV, Isaacs KL, et al. ACG Clinical Guideline: Management of\nCrohn\'s Disease in Adults. Am J Gastroenterol. 2018; 113(4):481-517.\n\n5\\. Ringold S, Angeles-Han ST, Beukelman T, et al. 2019 American College of\nRheumatology/Arthritis Foundation Guideline for the treatment of juvenile idiopathic\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\narthritis: therapeutic approaches for non-systemic polyarthritis, sacroiliitis, and\nenthesitis. Arthritis Rheumatol. 2019;71(6):846-863.\n\n6\\. Onel KB, Horton DB, Lovell DJ, et al. 2021 American College of Rheumatology guideline\nfor the treatment of juvenile idiopathic arthritis: therapeutic approaches for\noligoarthritis, temporomandibular joint arthritis, and systemic juvenile idiopathic\narthritis. Arthritis Rheumatol. 2022;74(4):553-569.\n\n7\\. Menter A, Strober BE, Kaplan DH, et al. Joint AAD-NPF guidelines of care for the\nmanagement and treatment of psoriasis with biologics. J Am Acad Dermatol.\n2019;80(4):1029-1072.\n\n8\\. Fraenkel L, Bathon JM, England BR, et al. 2021 American College of Rheumatology\nguideline for the treatment of rheumatoid arthritis. Arthritis Rheumatol.\n2021;73(7):1108-1123.\n\n9\\. Rubin DT, Ananthakrishnan AN, Siegel CA, et al. ACG clinical guideline: ulcerative colitis\nin adults. Am J Gastroenterol. 2019;114(3):384-413.\n\n10\\. Feuerstein JD, Isaac s KL, Schneider Y, et al. AGA clinical practice guidelines on the\nmanagement of moderate to severe ulcerative colitis. Gastroenterology. 2020;\n158:1450-1461.\n\n11\\. Pardi DS, D\'Haens G, Shen B, et al. Clinical guidelines for the management of pouchitis.\nInflamm Bowel Dis. 2009;15(9):1424-1431.\n\n12\\. Levy-Clarke G, Jabs DA, Read RW, et al. Expert panel recommendations for the use of\nanti-tumor necrosis factor biologic agents in patients with ocular inflammatory disorders.\nOphthalmology. 2014;121(3):785-796.\n\n13\\. Hatemi G, Christensen R, Bang D, et al. 2018 update of the EULAR recommendations for\nthe management of Behcet\'s syndrome. Ann Rheum Dis. 2018;77(6):808-818.\n\n14\\. Dabade TS, Davis MD. Diagnosis, and treatment of the neutrophilic dermatoses\n(pyoderma gangrenosum, Sweet\'s syndrome). Dermatol Ther. 2011;24(2):273-284.\n\n15\\. Baughman RP, Valeyre D, Korsten P, et al. ERS clinical practice guidelines on treatment\nof sarcoidosis. Eur Respir J. 2021;58(6):2004079.\n\n16\\. Dejaco C, Singh YP, Perel P, et al. 2015 Recommendations for the management of\npolymyalgia rheumatica: a European League Against Rheumatism/American College of\nRheumatology Collaborative Initiative. Arthritis Rheumatol. 2015;67(10):2569-2580.\n\n17\\. Feuerstein JD, Ho EY, Shmidt E, et al. AGA clinical practice guidelines on the medical\nmanagement of moderate to severe luminal and perianal fistulizing Crohn\'s disease.\nGastroenterology. 2021;160(7):2496-2508.\n\n18\\. Angeles-Han ST, Ringold S, Beukelman T, et al. 2019 American College of\nRheumatology/Arthritis Foundation guideline for the screening, monitoring, and\ntreatment of juvenile idiopathic arthritis-associated uveitis. Arthritis Rheumatol.\n2019;71(6):864-877.\n\n19\\. Ingram JR, Collier F, Brown D, et al. British Association of Dermatologists guidelines for\nthe management of hidradenitis suppurativa (acne inversa) 2018. Br J Dermatol.\n2019;180(5):1009-1017.\n\n\n## HISTORY\n\n\n<table>\n<tr>\n<th>Type of Revision</th>\n<th>Summary of Changes</th>\n<th>Review Date</th>\n</tr>\n<tr>\n<td>Annual Revision</td>\n<td>No criteria changes.</td>\n<td>04/05/2023</td>\n</tr>\n<tr>\n<td>Selected Revision</td>\n<td>The following biosimilars were added to the policy: Abrilada, adalimumab-adaz, adalimumab-fkjp, Cyltezo, Hadlima, Hulio, Hyrimoz, Idacio, Yuflyma, and Yusimry. The criteria for these biosimilars are that same as the existing criteria for Adalimumab Products. There were no other changes to the criteria.</td>\n<td>07/05/2023</td>\n</tr>\n</table>\n\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\n\n<table>\n<tr>\n<td>Annual Revision</td>\n<td>Plaque Psoriasis: For a patient currently taking an adalimumab product, the timeframe for established on therapy was changed from 90 days to 3 months. Behcet\'s Disease: For a patient currently taking an adalimumab product, the timeframe for established on therapy was changed from 90 days to 3 months. Hidradenitis Suppurativa: For a patient currently taking an adalimumab product, the timeframe for established on therapy was changed from 90 days to 3 months. Sarcoidosis: For a patient currently taking an adalimumab product, the timeframe for established on therapy was changed from 90 days to 3 months.</td>\n<td>03/27/2024</td>\n</tr>\n<tr>\n<td>Selected Revision</td>\n<td>Simlandi was added to the policy. The criteria for Simlandi are the same as the existing criteria for other adalimumab products. There are no other changes to the criteria.</td>\n<td>04/03/2024</td>\n</tr>\n<tr>\n<td>Selected Revision</td>\n<td>Adalimumab-aacf and adalimumab-ryvk were added to the policy. The criteria are the same as the existing criteria for other adalimumab products. The following changes were also made: Ankylosing Spondylitis: For initial approvals, a requirement that the patient is ≥ 18 years of age was added. Hidradenitis Suppurativa: For initial approvals, a requirement that the patient is ≥ 12 years of age was added. Juvenile Idiopathic Arthritis: For initial approvals, a requirement that the patient is ≥ 2 years of age was added. Plaque Psoriasis: In the Note, psoralen plus ultraviolet A light (PUVA) was removed from the examples of traditional systemic therapies. An additional Note was added that a 3-month trial of PUVA counts as a traditional systemic therapy. Psoriatic Arthritis: For initial approvals, a requirement that the patient is ≥ 18 years of age was added. Rheumatoid Arthritis: For initial approvals, a requirement that the patient is ≥ 18 years of age was added. Uveitis: For initial approvals, a requirement that the patient is ≥ 2 years of age was added. Behcet\'s Disease: For initial approvals, a requirement that the patient is ≥ 2 years of age was added. Pyoderma Gangrenosum: For initial approvals, a requirement that the patient is ≥ 18 years of age was added. Sarcoidosis: For initial approvals, a requirement that the patient is ≥ 18 years of age was added. Scleritis or Sterile Corneal Ulceration: For initial approvals, a requirement that the patient is ≥ 18 years of age was added. Spondyloarthritis, Other Subtypes: For initial approvals, a requirement that the patient is ≥ 18 years of age was added. Conditions Not Covered : Concurrent use with a Biologic or with a Targeted Synthetic Oral Small Molecule Drug was changed to as listed (previously oral small molecule drug was listed as Disease-Modifying Antirheumatic Drug).</td>\n<td>09/11/2024</td>\n</tr>\n</table>\n\n\n### APPENDIX\n\n\n<table>\n<tr>\n<th>Mechanism of</th>\n<th>Examples of</th>\n</tr>\n<tr>\n<td>Action</td>\n<td>Indications*</td>\n</tr>\n</table>\n\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\n\n<table>\n<tr>\n<td colspan="3">Biologics</td>\n</tr>\n<tr>\n<td>Adalimumab SC Products (Humira®, biosimilars)</td>\n<td>Inhibition of TNF</td>\n<td>AS, CD, JIA, PsO, PsA, RA, UC</td>\n</tr>\n<tr>\n<td>Cimzia® (certolizumab pegol SC injection)</td>\n<td>Inhibition of TNF</td>\n<td>AS, CD, nr-axSpA, PsO, PSA, RA</td>\n</tr>\n<tr>\n<td>Etanercept SC Products (Enbrel®, biosimilars)</td>\n<td>Inhibition of TNF</td>\n<td>AS, JIA, PsO, PSA, RA</td>\n</tr>\n<tr>\n<td>Infliximab IV Products (Remicade®, biosimilars)</td>\n<td>Inhibition of TNF</td>\n<td>AS, CD, PsO, PsA, RA, UC</td>\n</tr>\n<tr>\n<td>Zymfentra® (infliximab-dyyb SC injection)</td>\n<td>Inhibition of TNF</td>\n<td>CD, UC</td>\n</tr>\n<tr>\n<td rowspan="2">Simponi®, Simponi Aria® (golimumab SC injection, golimumab IV infusion)</td>\n<td rowspan="2">Inhibition of TNF</td>\n<td>SC formulation: AS, PsA, RA, UC</td>\n</tr>\n<tr>\n<td>IV formulation: AS, PJIA, PSA, RA</td>\n</tr>\n<tr>\n<td rowspan="2">Tocilizumab Products (Actemra® IV, biosimilar; Actemra SC, biosimilar)</td>\n<td rowspan="2">Inhibition of IL-6</td>\n<td>SC formulation: PJIA, RA, SJIA</td>\n</tr>\n<tr>\n<td>IV formulation: PJIA, RA, SJIA</td>\n</tr>\n<tr>\n<td>Kevzara® (sarilumab SC injection)</td>\n<td>Inhibition of IL-6</td>\n<td>RA</td>\n</tr>\n<tr>\n<td rowspan="2">Orencia® (abatacept IV infusion, abatacept SC injection)</td>\n<td rowspan="2">T-cell costimulation modulator</td>\n<td>SC formulation: JIA, PSA, RA</td>\n</tr>\n<tr>\n<td>IV formulation: JIA, PsA, RA</td>\n</tr>\n<tr>\n<td>Rituximab IV Products (Rituxan®, biosimilars)</td>\n<td>CD20-directed cytolytic antibody</td>\n<td>RA</td>\n</tr>\n<tr>\n<td>Kineret® (anakinra SC injection)</td>\n<td>Inhibition of IL-1</td>\n<td>JIA^, RA</td>\n</tr>\n<tr>\n<td>Omvoh® (mirikizumab IV infusion, SC injection)</td>\n<td>Inhibition of IL-23</td>\n<td>UC</td>\n</tr>\n<tr>\n<td rowspan="2">Stelara® (ustekinumab SC injection, ustekinumab IV infusion)</td>\n<td rowspan="2">Inhibition of IL- 12/23</td>\n<td>SC formulation: CD, PsO, PsA, UC</td>\n</tr>\n<tr>\n<td>IV formulation: CD, UC</td>\n</tr>\n<tr>\n<td>Siliq® (brodalumab SC injection)</td>\n<td>Inhibition of IL-17</td>\n<td>PsO</td>\n</tr>\n<tr>\n<td rowspan="2">Cosentyx® (secukinumab SC injection; secukinumab IV infusion)</td>\n<td>Inhibition of IL-17A</td>\n<td>SC formulation: AS, ERA, nr-axSpA, PsO, PsA</td>\n</tr>\n<tr>\n<td></td>\n<td>IV formulation: AS, nr- axSpA, PsA</td>\n</tr>\n<tr>\n<td>Taltz® (ixekizumab SC injection)</td>\n<td>Inhibition of IL-17A</td>\n<td>AS, nr-axSpA, PsO, PsA</td>\n</tr>\n<tr>\n<td>Bimzelx® (bimekizumab-bkzx SC injection)</td>\n<td>Inhibition of IL- 17A/17F</td>\n<td>PsO</td>\n</tr>\n<tr>\n<td>Ilumya® (tildrakizumab-asmn SC injection)</td>\n<td>Inhibition of IL-23</td>\n<td>PsO</td>\n</tr>\n<tr>\n<td rowspan="2">Skyrizi® (risankizumab-rzaa SC injection, risankizumab-rzaa IV infusion)</td>\n<td>Inhibition of IL-23</td>\n<td>SC formulation: CD, PSA, PsO, UC</td>\n</tr>\n<tr>\n<td></td>\n<td>IV formulation: CD, UC</td>\n</tr>\n<tr>\n<td rowspan="2">Tremfya® (guselkumab SC injection, guselkumab IV infusion)</td>\n<td>Inhibition of IL-23</td>\n<td>SC formulation: PsA, PsO, UC</td>\n</tr>\n<tr>\n<td></td>\n<td>IV formulation: UC</td>\n</tr>\n<tr>\n<td>Entyvio® (vedolizumab IV infusion, vedolizumab SC injection)</td>\n<td>Integrin receptor antagonist</td>\n<td>CD, UC</td>\n</tr>\n</table>\n\n\n## APPENDIX (CONTINUED)\n\n3 Pages - Cigna National Formulary Coverage - Policy:Inflammatory Conditions - Adalimumab Products Prior\nAuthorization Policy\n\n<!-- PageBreak -->\n\n\n<table>\n<tr>\n<th colspan="3">Oral Therapies/Targeted Synthetic Oral Small Molecule Drugs</th>\n</tr>\n<tr>\n<th>Otezla® (apremilast tablets)</th>\n<th>Inhibition of PDE4</th>\n<th>PsO, PsA</th>\n</tr>\n<tr>\n<td>Cibinqo™ (abrocitinib tablets)</td>\n<td>Inhibition of JAK pathways</td>\n<td>AD</td>\n</tr>\n<tr>\n<td>Olumiant® (baricitinib tablets)</td>\n<td>Inhibition of JAK pathways</td>\n<td>RA, AA</td>\n</tr>\n<tr>\n<td>Litfulo® (ritlecitinib capsules)</td>\n<td>Inhibition of JAK pathways</td>\n<td>AA</td>\n</tr>\n<tr>\n<td>Leqselvi® (deuruxolitinib tablets)</td>\n<td>Inhibition of JAK pathways</td>\n<td>AA</td>\n</tr>\n<tr>\n<td>Rinvoq® (upadacitinib extended- release tablets)</td>\n<td>Inhibition of JAK pathways</td>\n<td>AD, AS, nr-axSpA, RA, PsA, UC</td>\n</tr>\n<tr>\n<td>Rinvoq® LQ (upadacitinib oral solution)</td>\n<td>Inhibition of JAK pathways</td>\n<td>PSA, PJIA</td>\n</tr>\n<tr>\n<td>Sotyktu® (deucravacitinib tablets)</td>\n<td>Inhibition of TYK2</td>\n<td>PsO</td>\n</tr>\n<tr>\n<td>Xeljanz® (tofacitinib tablets/oral solution)</td>\n<td>Inhibition of JAK pathways</td>\n<td>RA, PJIA, PsA, UC</td>\n</tr>\n<tr>\n<td>Xeljanz® XR (tofacitinib extended- release tablets)</td>\n<td>Inhibition of JAK pathways</td>\n<td>RA, PsA, UC</td>\n</tr>\n<tr>\n<td>Zeposia® (ozanimod tablets)</td>\n<td>Sphingosine 1 phosphate receptor modulator</td>\n<td>UC</td>\n</tr>\n<tr>\n<td>Velsipity® (etrasimod tablets)</td>\n<td>Sphingosine 1 phosphate receptor modulator</td>\n<td>UC</td>\n</tr>\n</table>\n\n\\*\nNot an all-inclusive list of indications. Refer to the prescribing information for the\nrespective agent for FDA-approved indications; SC - Subcutaneous; TNF - Tumor necrosis\nfactor; AS - Ankylosing spondylitis; CD - Crohn\'s disease; JIA - Juvenile idiopathic arthritis;\nPsO - Plaque psoriasis; PsA - Psoriatic arthritis; RA - Rheumatoid arthritis; UC - Ulcerative\ncolitis; nr-axSpA - Non-radiographic axial spondyloarthritis; IV - Intravenous, PJIA -\nPolyarticular juvenile idiopathic arthritis; IL - Interleukin; SJIA - Systemic juvenile\nidiopathic arthritis; ^ Off-label use of Kineret in JIA supported in guidelines; ERA -\nEnthesitis-related arthritis; DMARD - Disease-modifying antirheumatic drug; PDE4 -\nPhosphodiesterase 4; JAK - Janus kinase; AD - Atopic dermatitis; AA - Alopecia areata;\nTYK2 - Tyrosine kinase 2.\n\n"Cigna Companies" refers to operating subsidiaries of The Cigna Group. All products and\nservices are provided exclusively by or through such operating subsidiaries, including Cigna\nHealth and Life Insurance Company, Connecticut General Life Insurance Company,\nEvernorth Behavioral Health, Inc., Cigna Health Management, Inc., and HMO or service\ncompany subsidiaries of The Cigna Group. @ 2024 The Cigna Group.\n\n\n<!-- PageFooter="3 Pages - Cigna National Formulary Coverage - Policy: Inflammatory Conditions - Adalimumab Products Prior Authorization Policy" -->\n'''

In [5]:
from src.pipeline.promptEngineering.prompt_manager import PromptManager
from src.aoai.aoai_helper import AzureOpenAIManager
from src.pipeline.autoDetermination.run import AutoPADeterminator
from src.pipeline.autoDetermination.evaluator import AutoDeterminationEvaluator
import os

azure_openai_client = AzureOpenAIManager(api_version=os.getenv("AZURE_OPENAI_API_VERSION_01"))
prompt_manager = PromptManager()
evaluator = AutoDeterminationEvaluator(cases_dir="../../evals/cases", temp_dir="./temp_evaluation_rag", config_file_path="../../../.azure/config.json")
auto_pa_determinator = AutoPADeterminator(
            azure_openai_client=azure_openai_client,
            azure_openai_client_o1=azure_openai_client,
            prompt_manager=prompt_manager,
            caseId='001',
        )
evaluator.auto_determinator = auto_pa_determinator

2025-04-08 22:02:34,959 - AIFoundryManager - MainProcess - INFO     Configuration validation successful. (aifoundry_helper.py:_validate_configurations:58)
2025-04-08 22:02:34,960 - AIFoundryManager - MainProcess - INFO     AIProjectClient initialized successfully. (aifoundry_helper.py:_initialize_project:103)


Templates found: ['evaluator_system_prompt.jinja', 'evaluator_user_prompt.jinja', 'formulator_system_prompt.jinja', 'formulator_user_prompt.jinja', 'ner_clinician_system.jinja', 'ner_clinician_user.jinja', 'ner_patient_system.jinja', 'ner_patient_user.jinja', 'ner_physician_system.jinja', 'ner_physician_user.jinja', 'ner_system_prompt.jinja', 'ner_user_prompt.jinja', 'prior_auth_o1_user_prompt.jinja', 'prior_auth_o1_user_prompt_b.jinja', 'prior_auth_system_prompt.jinja', 'prior_auth_user_prompt.jinja', 'query_classificator_system_prompt.jinja', 'query_classificator_user_prompt.jinja', 'query_expansion_system_prompt.jinja', 'query_expansion_user_prompt.jinja', 'summarize_autodetermination_system.jinja', 'summarize_autodetermination_user.jinja', 'summarize_policy_system.jinja', 'summarize_policy_user.jinja', 'transform_determination_markdown_system_prompt.jinja', 'transform_determination_markdown_user_prompt.jinja']
Environment variables loaded from: ../../../.azure/autoauth-ui-fix/.env


In [10]:
import nest_asyncio
nest_asyncio.apply()

response = await evaluator.generate_responses(patient_info, physician_info, clinical_info, policy_text)
structured_output = await evaluator._summarize_autodetermination(response)

2025-04-08 22:06:03,036 - autoDetermination - MainProcess - INFO     Generating final determination for None (run.py:run:102)
2025-04-08 22:06:03,037 - autoDetermination - MainProcess - INFO     Input clinical information: **Task:**

Analyze a prior authorization request for a medical treatment or medication using the provided patient information, physician information, clinical information, and policy text.

---

**Purpose:**

Determine whether the prior authorization request should be Approved, Denied, or if More Information is Needed.

---

**Approach:**

- Analyze all policy criteria by thoroughly reviewing the policy text.
- Extract necessary information from the policy and the provided data.
- Compare the extracted information from the patient, physician, and clinical data against the policy criteria.
- Make a detailed decision based on these comparisons.

---

**Instructions:**

**Analyze Policy Criteria:**

- Carefully read the entire policy text relevant to the requested medic

In [18]:
result = json.loads(structured_output)
result

{'Determination': 'Approved',
 'Rationale': 'Based on the policy’s criteria for Crohn’s Disease in patients age 6 or older with prior corticosteroid use, and the prescribing physician being a pediatric gastroenterologist, the request meets all requirements for approval.',
 'DetailedAnalysis': {'PolicyCriteriaAssessment': [{'CriterionName': 'Patient is ≥ 6 years of age (Crohn’s Disease – Initial Therapy requirement)',
    'Assessment': 'Fully Met',
    'Evidence': 'Patient’s date of birth is 10/19/2014 (age 9).',
    'PolicyReference': 'Crohn’s Disease. Approve… if the patient is ≥ 6 years of age.'},
   {'CriterionName': 'Patient has tried or is currently taking corticosteroids (Crohn’s Disease – Initial Therapy requirement)',
    'Assessment': 'Fully Met',
    'Evidence': 'The patient has been on methylprednisolone (40 mg daily) with no improvement.',
    'PolicyReference': 'Patient has tried or is currently taking corticosteroids…'},
   {'CriterionName': 'The medication is prescribed 

In [21]:
from src.evals.custom.fuzzy_evaluator import FuzzyEvaluator

fuzzy_evaluator = FuzzyEvaluator()
fuzzy_evaluator(response=result['Determination'], ground_truth="Approved")

IndelSimilarity(indel_similarity=100.0)

In [22]:
fuzzy_evaluator(response=result['Determination'], ground_truth="Rejected")

IndelSimilarity(indel_similarity=25.0)

In [24]:
fuzzy_evaluator(response=result['Determination'], ground_truth="approved")

IndelSimilarity(indel_similarity=87.5)

In [26]:
fuzzy_evaluator(response=str(
        len([x for x in result['DetailedAnalysis']['PolicyCriteriaAssessment'] if x['Assessment'] != 'Fully Met']) == 0
    ), 
    ground_truth="True")

IndelSimilarity(indel_similarity=100.0)

In [64]:
from azure.ai.evaluation import evaluate, SimilarityEvaluator

model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_ID"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
}

gpt_similarity_evaluator = SimilarityEvaluator(model_config=model_config)
gpt_similarity_evaluator(query="Approval for Prior Authorization", response=result['Rationale'], ground_truth="Based on the provided information, Sarah Sample meets all the policy criteria for initial therapy approval of adalimumab for Crohn’s Disease. She is over 6 years old, has tried corticosteroid therapy without improvement, and the medication is prescribed by a pediatric gastroenterologist.")


{'similarity': 5.0, 'gpt_similarity': 5.0}

In [37]:
gpt_similarity_evaluator(query="Approval for Prior Authorization", response=result['Rationale'], ground_truth="Sarah is rejected based on the criteria.")

{'similarity': 1.0, 'gpt_similarity': 1.0}

In [38]:
gpt_similarity_evaluator(query="Approval for Prior Authorization", response=result['Rationale'], ground_truth="Sarah is approved based on the criteria.")

{'similarity': 3.0, 'gpt_similarity': 3.0}

In [34]:
from src.evals.custom.factual_correctness_evaluator import FactualCorrectnessEvaluator

factual_correctness = FactualCorrectnessEvaluator(model_config=model_config)
factual_correctness(response=result['Rationale'], ground_truth="Based on the provided information, Sarah Sample meets all the policy criteria for initial therapy approval of adalimumab for Crohn’s Disease. She is over 6 years old, has tried corticosteroid therapy without improvement, and the medication is prescribed by a pediatric gastroenterologist.")

/Users/marcjimz/Documents/Development/autoauth-solution-accelerator/src/evals/custom/factual_correctness_evaluator.py:52: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  azure_llm = AzureChatOpenAI(


{'factual_correctness': 0.73}

In [39]:
factual_correctness(response=result['Rationale'], ground_truth="Sarah is rejected based on the criteria.")

{'factual_correctness': 0.0}

In [40]:
factual_correctness(response=result['Rationale'], ground_truth="Sarah is approved based on the criteria.")

{'factual_correctness': 0.33}

In [41]:
from src.evals.custom.similarity_evaluator import SemanticSimilarityEvaluator

semantic_similarity = SemanticSimilarityEvaluator()
semantic_similarity(response=result['Rationale'], ground_truth="Based on the provided information, Sarah Sample meets all the policy criteria for initial therapy approval of adalimumab for Crohn’s Disease. She is over 6 years old, has tried corticosteroid therapy without improvement, and the medication is prescribed by a pediatric gastroenterologist.")

{'semantic_similarity': 0.9072816371917725}

In [46]:
semantic_similarity(response=result['Rationale'], ground_truth="Sarah is rejected.")

{'semantic_similarity': 0.3547055721282959}

In [45]:
semantic_similarity(response=result['Rationale'], ground_truth="Sarah is approved.")

{'semantic_similarity': 0.4083213210105896}

In [47]:
response = await evaluator.generate_responses(patient_info, None, clinical_info, policy_text)
structured_output = await evaluator._summarize_autodetermination(response)

2025-04-08 22:34:18,504 - autoDetermination - MainProcess - ERROR    Error generating response for autoDetermination: 'NoneType' object has no attribute 'physician_name' (evaluator.py:generate_responses:282)
2025-04-08 22:34:18,505 - autoDetermination - MainProcess - INFO     Summarizing AutoDetermination... (evaluator.py:_summarize_autodetermination:338)
2025-04-08 22:34:18,506 - micro - MainProcess - INFO     Function generate_chat_response started at 2025-04-08 22:34:18 (aoai_helper.py:generate_chat_response:383)
2025-04-08 22:34:18,506 - micro - MainProcess - INFO     Sending request to Azure OpenAI at 2025-04-08 22:34:18 (aoai_helper.py:generate_chat_response:440)
2025-04-08 22:34:20,210 - micro - MainProcess - INFO     Function generate_chat_response finished at 2025-04-08 22:34:20 (Duration: 1.70 seconds) (aoai_helper.py:generate_chat_response:494)


In [49]:
json.loads(structured_output)

{'Determination': 'Needs More Information',
 'Rationale': "The adjudication could not be completed due to missing information regarding the physician's name, which is required to verify the request against policy criteria.",
 'DetailedAnalysis': {'PolicyCriteriaAssessment': [],
  'MissingInformation': [{'InformationNeeded': 'Physician Name',
    'Reason': "The physician's name is necessary to verify the credentials and ensure compliance with policy requirements for authorization."}]}}

In [51]:
physician_info_incorrect = physician_info
physician_info_incorrect.specialty = "Pediatrics"
response = await evaluator.generate_responses(patient_info, physician_info_incorrect, clinical_info, policy_text)
structured_output = await evaluator._summarize_autodetermination(response)

2025-04-08 22:35:57,812 - autoDetermination - MainProcess - INFO     Generating final determination for None (run.py:run:102)
2025-04-08 22:35:57,813 - autoDetermination - MainProcess - INFO     Input clinical information: **Task:**

Analyze a prior authorization request for a medical treatment or medication using the provided patient information, physician information, clinical information, and policy text.

---

**Purpose:**

Determine whether the prior authorization request should be Approved, Denied, or if More Information is Needed.

---

**Approach:**

- Analyze all policy criteria by thoroughly reviewing the policy text.
- Extract necessary information from the policy and the provided data.
- Compare the extracted information from the patient, physician, and clinical data against the policy criteria.
- Make a detailed decision based on these comparisons.

---

**Instructions:**

**Analyze Policy Criteria:**

- Carefully read the entire policy text relevant to the requested medic

{'Determination': 'Rejected', 'Rationale': "The patient meets most requirements for adalimumab coverage for Crohn’s disease, such as age and steroid trial. However, the policy requires the prescribing physician to be a gastroenterologist or to have direct consultation with one. The provided information only indicates a pediatric specialist, with no mention of a gastroenterologist's involvement. Therefore, the prescriber specialty/consultation requirement is not satisfied, leading to denial of coverage.", 'DetailedAnalysis': {'PolicyCriteriaAssessment': [{'CriterionName': 'Patient Age ≥ 6 Years', 'Assessment': 'Fully Met', 'Evidence': 'The patient’s date of birth is 10/19/2014, making her 9 years old.', 'PolicyReference': 'Crohn’s Disease … Patient is ≥ 6 years of age.'}, {'CriterionName': 'Use of Corticosteroids or Other Conventional Therapy', 'Assessment': 'Fully Met', 'Evidence': 'The patient has been on methylprednisolone 40 mg daily with no improvement.', 'PolicyReference': 'Crohn’

In [53]:
json.loads(structured_output)

{'Determination': 'Rejected',
 'Rationale': 'The request does not meet a key policy requirement that adalimumab for Crohn’s disease be prescribed by or in consultation with a gastroenterologist. The prescribing physician’s specialty is listed only as Pediatrics, and there is no documentation of consultation with a gastroenterologist. According to the policy, if a required criterion is not demonstrated, the request must be denied.',
 'DetailedAnalysis': {'PolicyCriteriaAssessment': [{'CriterionName': 'Age Requirement (≥ 6 years for Crohn’s Disease)',
    'Assessment': 'Fully Met',
    'Evidence': 'Patient is 9 years old (DOB 10/19/2014).',
    'PolicyReference': 'Crohn’s Disease. … Patient is ≥ 6 years of age.'},
   {'CriterionName': 'Prior Therapy with Corticosteroids or Conventional Systemic Treatments',
    'Assessment': 'Fully Met',
    'Evidence': 'Patient has been on methylprednisolone 40 mg daily without improvement.',
    'PolicyReference': 'Patient has tried or is currently tak

In [61]:
evaluation_data = [
    {
        "query" : "001-Determination",
        "ground_truth" : json.loads(structured_output)['Determination'],
        "response" : "Negative",
        "context" : json.dumps({'physician_info' : physician_info.model_dump(),
                                'clinical_info' : clinical_info.model_dump(),
                                'policy_text' : policy_text,
                                'patient_info' : patient_info.model_dump()})
    },
    {
        "query" : "001-Rationale",
        "ground_truth" : json.loads(structured_output)['Rationale'],
        "response" : "The request does not meet a key policy requirement that adalimumab for Crohn’s disease be prescribed by or in consultation with a gastroenterologist. The prescribing physician’s specialty is listed only as Pediatrics, and there is no documentation of consultation with a gastroenterologist. According to the policy, if a required criterion is not demonstrated, the request must be denied.",
        "context" : json.dumps({'physician_info' : physician_info.model_dump(),
                                'clinical_info' : clinical_info.model_dump(),
                                'policy_text' : policy_text,
                                'patient_info' : patient_info.model_dump()})
    }
]

In [63]:
# Write the evaluation dataset to a temporary JSONL file.
temp_dataset = "temp_dataset.jsonl"
with open(temp_dataset, "w", encoding="utf8") as f:
    for item in evaluation_data:  # Limit to first 100 items for testing
        f.write(json.dumps(item) + "\n")

In [66]:
from src.aifoundry.aifoundry_helper import AIFoundryManager

evaluators = {
    "fuzzy_evaluator": fuzzy_evaluator,
    "semantic_similarity": semantic_similarity,
    "gpt_similarity_evaluator": gpt_similarity_evaluator,
    "factual_correctness": factual_correctness,
}

evaluator_config = {
    "fuzzy_evaluator": {"column_mapping": {"response": "${data.response}", "ground_truth": "${data.ground_truth}", "query": "${data.query}"}},
    "semantic_similarity": {"column_mapping": {"response": "${data.response}", "ground_truth": "${data.ground_truth}", "query": "${data.query}"}},
    "factual_correctness": {"column_mapping": {"response": "${data.response}",  "ground_truth": "${data.ground_truth}", "query": "${data.query}"}},
    "gpt_similarity_evaluator": {"column_mapping": {"response": "${data.response}",  "ground_truth": "${data.ground_truth}", "query": "${data.query}"}}
}

evaluation_name = f"Notebook_Evaluation_Submission_001"

# Run evaluation using the Azure AI Evaluation SDK.
ai_foundry_manager = AIFoundryManager()
results = evaluate(
    evaluation_name=evaluation_name,
    data=temp_dataset,
    evaluators=evaluators,
    evaluator_config=evaluator_config,
    azure_ai_project=ai_foundry_manager.project_config,
)

2025-04-08 22:51:26,991 - AIFoundryManager - MainProcess - INFO     Configuration validation successful. (aifoundry_helper.py:_validate_configurations:58)
2025-04-08 22:51:26,992 - AIFoundryManager - MainProcess - INFO     AIProjectClient initialized successfully. (aifoundry_helper.py:_initialize_project:103)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current proce

2025-04-08 22:51:27 -0700   81705 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-04-08 22:51:28 -0700   81705 execution.bulk     INFO     Finished 1 / 2 lines.
2025-04-08 22:51:28 -0700   81705 execution.bulk     INFO     Average execution time for completed lines: 0.8 seconds. Estimated time for incomplete lines: 0.8 seconds.
2025-04-08 22:51:28 -0700   81705 execution.bulk     INFO     Finished 2 / 2 lines.
2025-04-08 22:51:28 -0700   81705 execution.bulk     INFO     Average execution time for completed lines: 0.41 seconds. Estimated time for incomplete lines: 0.0 seconds.
======= Run Summary =======

Run name: "azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_y82b37_z_20250408_225127_041714"
Run status: "Completed"
Start time: "2025-04-08 22:51:27.039782-07:00"
Duration: "0:00:01.419915"
Output path: "/Users/marcjimz/.promptflow/.runs/azure_ai_evaluation_evaluators_common_base_eval_asyncevaluato

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2025-04-08 22:51:34 -0700   81705 execution.bulk     INFO     Process name(SpawnProcess-4)-Process id(84152)-Line number(1) completed.
2025-04-08 22:51:34 -0700   81705 execution.bulk     INFO     Process name(SpawnProcess-7)-Process id(84158)-Line number(0) completed.


/Users/marcjimz/Documents/Development/autoauth-solution-accelerator/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/marcjimz/Documents/Development/autoauth-solution-accelerator/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/marcjimz/Documents/Development/autoauth-solution-accelerator/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['p

2025-04-08 22:51:35 -0700   81705 execution.bulk     INFO     Finished 2 / 2 lines.
2025-04-08 22:51:35 -0700   81705 execution.bulk     INFO     Average execution time for completed lines: 2.0 seconds. Estimated time for incomplete lines: 0.0 seconds.
2025-04-08 22:51:35 -0700   81705 execution.bulk     INFO     The thread monitoring the process [84152-SpawnProcess-4] will be terminated.
2025-04-08 22:51:35 -0700   81705 execution.bulk     INFO     The thread monitoring the process [84158-SpawnProcess-7] will be terminated.
2025-04-08 22:51:35 -0700   81705 execution.bulk     INFO     Process 84158 terminated.
2025-04-08 22:51:35 -0700   81705 execution.bulk     INFO     Process 84152 terminated.
2025-04-08 22:51:27 -0700   81705 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-04-08 22:51:27 -0700   81705 execution.bulk     INFO     The timeout for the batch run is 3600 seconds.
2025-04-08 22:51:27 -0700   81705 exec

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2025-04-08 22:51:47 -0700   81705 execution.bulk     INFO     Process name(SpawnProcess-6)-Process id(84154)-Line number(1) completed.
2025-04-08 22:51:47 -0700   81705 execution.bulk     INFO     Finished 2 / 2 lines.
2025-04-08 22:51:47 -0700   81705 execution.bulk     INFO     Average execution time for completed lines: 8.01 seconds. Estimated time for incomplete lines: 0.0 seconds.
2025-04-08 22:51:47 -0700   81705 execution.bulk     INFO     The thread monitoring the process [84159-SpawnProcess-8] will be terminated.
2025-04-08 22:51:47 -0700   81705 execution.bulk     INFO     The thread monitoring the process [84154-SpawnProcess-6] will be terminated.
2025-04-08 22:51:48 -0700   81705 execution.bulk     INFO     Process 84159 terminated.
2025-04-08 22:51:48 -0700   81705 execution.bulk     INFO     Process 84154 terminated.
2025-04-08 22:51:27 -0700   81705 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-04-08 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


======= Combined Run Summary (Per Evaluator) =======

{
    "fuzzy_evaluator": {
        "status": "Completed",
        "duration": "0:00:08.917991",
        "completed_lines": 2,
        "failed_lines": 0,
        "log_path": "/Users/marcjimz/.promptflow/.runs/src_evals_custom_fuzzy_evaluator_fuzzyevaluator_eic7cenk_20250408_225127_042902"
    },
    "semantic_similarity": {
        "status": "Completed",
        "duration": "0:00:23.192279",
        "completed_lines": 2,
        "failed_lines": 0,
        "log_path": "/Users/marcjimz/.promptflow/.runs/src_evals_custom_similarity_evaluator_semanticsimilarityevaluator_hy_vfysl_20250408_225127_042093"
    },
    "gpt_similarity_evaluator": {
        "status": "Completed",
        "duration": "0:00:01.419915",
        "completed_lines": 2,
        "failed_lines": 0,
        "log_path": "/Users/marcjimz/.promptflow/.runs/azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_y82b37_z_20250408_225127_041714"
    },
    "factual

Now what if we want to replicate this across the board and make sure it is adhered to?

In [93]:
# Instantiate PatientInformation for "Lucas Little"
patient_info = PatientInformation(
    patient_name="Lucas Little",
    patient_date_of_birth="07/30/2017",
    patient_id="345987",
    patient_address="28 Dearborn St, Chicago, IL, 60602",
    patient_phone_number="555-360-8746"
)

# Instantiate PhysicianInformation for Dr. Oncoso
physician_info = PhysicianInformation(
    physician_name="Dr. Oncoso",
    specialty="Pediatric Hematology and Oncology",
    physician_contact={
        "office_phone": "555-324-7878",
        "fax": "555-324-7877",
        "office_address": "27 W State St, Chicago, IL 60601"
    }
)

# Instantiate ClinicalInformation with the detailed clinical context
clinical_info = ClinicalInformation(
    diagnosis="B-cell precursor acute lymphoblastic leukemia (B-ALL); Philadelphia chromosome-negative (Ph-); Minimal Residual Disease (MRD+)",
    icd_10_code="C91.00",
    prior_treatments_and_results=(
        "Vincristine: Administered weekly during induction phase; "
        "Dexamethasone: Administered daily for 28 days during induction phase; "
        "L-asparaginase: Administered on alternate days during induction phase; "
        "Daunorubicin: Given as a single dose during the first week of induction phase; "
        "Methotrexate: High-dose, given weekly during consolidation phase; "
        "Mercaptopurine: Daily oral administration during maintenance phase; "
        "Vincristine: Administered biweekly during maintenance phase; "
        "Cyclophosphamide: Given during the third week of maintenance phase; "
        "Doxorubicin: Administered during the final week of consolidation phase"
    ),
    specific_drugs_taken_and_failures=(
        "Vincristine: Administered weekly during induction phase; "
        "Dexamethasone: Administered daily for 28 days during induction phase; "
        "L-asparaginase: Administered on alternate days during induction phase; "
        "Daunorubicin: Given as a single dose during the first week of induction phase; "
        "Methotrexate: High-dose, given weekly during consolidation phase; "
        "Mercaptopurine: Daily oral administration during maintenance phase; "
        "Vincristine: Administered biweekly during maintenance phase; "
        "Cyclophosphamide: Given during the third week of maintenance phase; "
        "Doxorubicin: Administered during the final week of consolidation phase"
    ),
    alternative_drugs_required="Not provided",
    relevant_lab_results_or_imaging=(
        "Bone Marrow Biopsy (03/29/24): Hypercellular (approximately 90% cellularity), 85% lymphoblasts, Philadelphia chromosome-negative (Ph-); "
        "Bone Marrow Biopsy (10/4/24): Hypercellular (approximately 90% cellularity), Less than 5% blast cells, Positive for minimal residual disease (MRD+); "
        "Laboratory Results (10/4/24): WBC 4,500 - 10,000 cells/µL, Neutrophils 40%, Lymphocytes 20%, Monocytes 2%, "
        "Eosinophils 1%, Basophils 0%, Hemoglobin 11.5 g/dL, Hematocrit 35%, Platelet Count 150,000 - 450,000 cells/µL"
    ),
    symptom_severity_and_impact=(
        "Fatigue, decreased energy, mild fatigue and decreased energy, but overall well appearing with no recent fevers"
    ),
    prognosis_and_risk_if_not_approved=(
        "Presence of minimal residual disease suggests residual leukemic activity; Continuous monitoring and further therapeutic intervention may be warranted; "
        "Potential for disease progression if not treated"
    ),
    clinical_rationale_for_urgency=(
        "Applying the standard review time frame may seriously jeopardize the customer's life, health, or ability to regain maximum function"
    ),
    treatment_request={
        "name_of_medication_or_procedure": "Blinatumomab",
        "code_of_medication_or_procedure": "Not provided",
        "dosage": "28 mcg/day",
        "duration": (
            "Induction cycle 1: Days 1-28: 28 mcg/day; Based on clinical response, up to 6 months"
        ),
        "rationale": (
            "Management of MRD+ B-ALL; Patient has achieved clinical remission with less than 5% blast cells, "
            "but positive for minimal residual disease (MRD+)"
        ),
        "presumed_eligibility": "Not provided"
    }
)
policy_text='''          <figure>

          envolve
          Pharmacy Solutions

          </figure>


          # Clinical Policy: Blinatumomab (Blincyto)

          Reference Number: ERX.SPA.241

          Effective Date: 09.01.18

          Last Review Date: 08.18

          Revision Log

          See Important Reminder at the end of this policy for important regulatory and legal information.


          ## Description

          Blinatumomab (Blincyto®) is a bispecific CD19-directed CD3 T-cell engager that binds to CD19
          (expressed on cells of B-lineage origin) and CD3 (expressed on T cells).

          FDA Approved Indication(s)

          Blincyto is indicated for:

          · MRD-positive B-cell precursor ALL

          o Treatment of B-cell precursor acute lymphoblastic leukemia (ALL) in first or second complete
          ☐
          remission with minimal residual disease (MRD) greater than or equal to 0.1% in adults and
          children. This indication is approved under accelerated approval based on MRD response rate
          and hematological relapse-free survival. Continued approval for this indication may be contingent
          upon verification and description of clinical benefit in the confirmatory trials.

          · Relapsed or refractory B-cell precursor ALL

          o Treatment of relapsed or refractory B-cell precursor acute lymphoblastic leukemia (ALL) in adults
          ☐
          and children.

          Policy/Criteria

          Provider must submit documentation (such as office chart notes, lab results or other clinical information)
          supporting that member has met all approval criteria.

          It is the policy of health plans affiliated with Envolve Pharmacy Solutions ™ that Blincyto is medically
          necessary when the following criteria are met:


          ## I. Initial Approval Criteria

          A. Acute Lymphoblastic Leukemia (must meet all):

          1\. Diagnosis of B-cell precursor acute lymphoblastic leukemia (B-ALL);

          2\. Prescribed by or in consultation with an oncologist or hematologist;

          3\. Requested as treatment for (a or b):

          a. B-ALL in remission but positive for minimal residual disease (MRD+);

          b. Relapsed or refractory B-ALL (i and ii):

          i. Philadelphia chromosome-negative (Ph-) disease;

          ii. Philadelphia chromosome-positive (Ph+) disease and intolerant or refractory to at
          least one second-generation or later tyrosine kinase inhibitor (TKI; i.e., Sprycel®,
          Tasigna®, Bosulif®, Iclusig®);
          *Prior authorization is (or may be) required for these agents

          4\. Dose does not exceed 28 mcg/day.

          Approval duration: 6 months


          ### B. Other diagnoses/indications

          1\. Refer to ERX.PA.01 if diagnosis is NOT specifically listed under section III
          (Diagnoses/Indications for which coverage is NOT authorized).


          ## II. Continued Therapy

          A. Acute Lymphoblastic Leukemia (must meet all):

          1\. Currently receiving medication via a health plan affiliated with Envolve Pharmacy Solutions,
          or documentation supports that member is currently receiving Blincyto for a covered
          indication and has received this medication for at least 30 days;

          2\. Member is responding positively to therapy;

          <!-- PageNumber="Page 1 of 4" -->
          <!-- PageBreak -->


          <figure>

          envolve.
          Pharmacy Solutions

          TH

          </figure>


          <!-- PageHeader="CLINICAL POLICY Blinatumomab" -->

          3\. If request is for a dose increase, new dose does not exceed 28 mcg/day.
          Approval duration: 12 months

          B. Other diagnoses/indications (must meet 1 or 2):

          1\. Currently receiving medication via a health plan affiliated with Envolve Pharmacy Solutions
          and documentation supports positive response to therapy.
          Approval duration: Duration of request or 6 months (whichever is less); or

          2\. Refer to ERX.PA.01 if diagnosis is NOT specifically listed under section III
          (Diagnoses/Indications for which coverage is NOT authorized).


          ### III. Diagnoses/Indications for which coverage is NOT authorized:

          A. Non-FDA approved indications, which are not addressed in this policy, unless there is sufficient
          documentation of efficacy and safety according to the off-label use policy - ERX.PA.01 or
          evidence of coverage documents.


          ### IV. Appendices/General Information

          Appendix A: Abbreviation/Acronym Key

          B-ALL: B-cell precursor acute lymphoblastic
          leukemia
          CR: complete remission

          FDA: Food and Drug Administration
          MRD+: positive minimal residual disease
          TKI: tyrosine kinase inhibitor

          Appendix B: Therapeutic Alternatives
          This table provides a listing of preferred alternative therapy recommended in the approval criteria.
          The drugs listed here may not be a formulary agent and may require prior authorization.


          <table>
          <tr>
          <td>Drug Name</td>
          <td>Dosing Regimen</td>
          <td>Dose Limit/ Maximum Dose</td>
          </tr>
          <tr>
          <td>Sprycel® (dasatinib)</td>
          <td>140 mg PO QD (adults*)</td>
          <td>180 mg/day</td>
          </tr>
          <tr>
          <td>Iclusig® (ponatinib)</td>
          <td>45 mg PO QD (adults*)</td>
          <td>45 mg/day</td>
          </tr>
          <tr>
          <td>Tasigna® (nilotinib)</td>
          <td>400 mg PO BID (off-label use; adults* - as referenced in Kim, et al., 2015; see also Appendix D)</td>
          <td>800 mg/day</td>
          </tr>
          <tr>
          <td>Bosulif® (bosutinib)</td>
          <td>500 to 600 mg PO QD (off-label use; adults* - as referenced in Gambacroti-Passerini, et al., 2015; see also Appendix D).</td>
          <td>600 mg/day</td>
          </tr>
          </table>

          Therapeutic alternatives are listed as Brand name® (generic) when the drug is available by brand name only and generic
          (Brand name®) when the drug is available by both brand and generic.

          *Sprycel and Iclusig are FDA approved for Ph+ ALL in adults; however, all four listed TKIs are NCCN recommended (category
          2A) for Ph+ ALL in both adults and adolescents/young adults.


          Appendix C: Contraindications

          Not applicable


          #### Appendix D: General Information

          · MRD-positive B-cell precursor ALL

          o In 2018, Blincyto received FDA approval for MRD+ B-ALL in remission based on a single-
          arm, open label study (BLAST) showing complete MRD response in a majority of adults
          undergoing Blincyto therapy; the new FDA indication includes both children and adults based
          on this pivotal trial.

          · Relapsed or refractory B-cell precursor ALL

          o In 2017, blinatumomab's labeled use was expanded from treatment of Ph- relapsed/refractory
          B-ALL to treatment of Ph+ disease based on a single-arm, open label study (ALCANTARAA)
          showing complete remission (CR), or CR with partial hematologic recovery, after disease
          progression on at least one second- or thid-generation TKI. FDA approved second- and third-
          generation TKIs for Ph+ ALL in adults include Sprycel and Iclusig. NCCN recommended
          (category 2A) TKIs for Ph+ ALL in adults and adolescents/young adults include Sprycel,
          Iclusig, Tasigna and Bosulif.

          <!-- PageNumber="Page 2 of 4" -->
          <!-- PageBreak -->

          <!-- PageHeader="CLINICAL POLICY Blinatumomab" -->


          <figure>

          envolve®
          Pharmacy Solutions

          TM

          </figure>


          ## V. Dosage and Administration


          <table>
          <tr>
          <th>Indication</th>
          <th>Dosing Regimen</th>
          <th>Maximum Dose</th>
          </tr>
          <tr>
          <td>B-ALL (MRD+ in</td>
          <td>Treatment course: 1 cycle of Blincyto IV for induction followed by up to 3 additional cycles for consolidation.</td>
          <td>28 mcg/day</td>
          </tr>
          <tr>
          <td rowspan="7">remission)</td>
          <td>· Patients ≥ 45 kg receive a fixed dose</td>
          <td></td>
          </tr>
          <tr>
          <td>o Induction cycle 1</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 1-28: 28 mcg/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 29-42: 14-day treatment-free interval</td>
          <td></td>
          </tr>
          <tr>
          <td>o Consolidation cycles 2-4</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 1-28: 28 mcg/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 29-42: 14-day treatment-free interval</td>
          <td></td>
          </tr>
          <tr>
          <td rowspan="6"></td>
          <td>· Patients &lt; 45 kg based on body surface area (BSA)</td>
          <td></td>
          </tr>
          <tr>
          <td>o Induction cycle 1</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 1-7: 5 mcg/m2/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 8-28: 15 mcg/m2/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 29-42: 14-day treatment-free interval</td>
          <td></td>
          </tr>
          <tr>
          <td>o Consolidation cycles 2-4</td>
          <td></td>
          </tr>
          <tr>
          <td rowspan="2"></td>
          <td>· Days 1-28: 15 mcg/m2/day</td>
          <td></td>
          </tr>
          <tr>
          <td>Days 29-42: 14-day treatment-free interval</td>
          <td></td>
          </tr>
          <tr>
          <td rowspan="2">B-ALL (relapsed</td>
          <td rowspan="3">Treatment course: 2 cycles of Blincyto IV for induction followed by 3 cycles for consolidation and up to 4 cycles of continued therapy.</td>
          <td>28 mcg/day</td>
          </tr>
          <tr>
          <td></td>
          </tr>
          <tr>
          <td>or</td>
          <td></td>
          </tr>
          <tr>
          <td rowspan="5">refractory)</td>
          <td>· Patients ≥ 45 kg receive a fixed dose</td>
          <td></td>
          </tr>
          <tr>
          <td>o Induction cycle 1</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 1-7: 9 mcg/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 8-28: 28 mcg/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 29-42: 14-day treatment-free interval</td>
          <td></td>
          </tr>
          <tr>
          <td rowspan="9"></td>
          <td>o Induction cycle 2</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 1-28: 28 mcg/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 29-42: 14-day treatment-free interval</td>
          <td></td>
          </tr>
          <tr>
          <td>o Consolidation cycles 3-5</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 1-28: 28 mcg/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 29-42: 14-day treatment-free interval</td>
          <td></td>
          </tr>
          <tr>
          <td>o Continued therapy cycles 6-9</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 1-28: 28 mcg/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 29-84: 56-day treatment-free interval</td>
          <td></td>
          </tr>
          <tr>
          <td rowspan="13"></td>
          <td>· Patients &lt; 45 kg based on body surface area (BSA)</td>
          <td></td>
          </tr>
          <tr>
          <td>o Induction cycle 1</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 1-7: 5 mcg/m2/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 8-28: 15 mcg/m2/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 29-42: 14-day treatment-free interval</td>
          <td></td>
          </tr>
          <tr>
          <td>o Induction cycle 2</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 1-28: 15 mcg/m2/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 29-42: 14-day treatment-free interval</td>
          <td></td>
          </tr>
          <tr>
          <td>o Consolidation cycles 3-5</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 1-28: 15 mcg/m2/day</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 29-42: 14-day treatment-free interval</td>
          <td></td>
          </tr>
          <tr>
          <td>o Continued therapy cycles 6-9</td>
          <td></td>
          </tr>
          <tr>
          <td>· Days 1-28: 15 mcg/m2/day</td>
          <td></td>
          </tr>
          <tr>
          <td></td>
          <td>Days 29-84: 56-day treatment-free interval</td>
          <td></td>
          </tr>
          </table>


          ## VI. Product Availability Single-dose vial for reconstitution: 35 mcg


          ### VII. References

          <!-- PageNumber="Page 3 of 4" -->
          <!-- PageBreak -->


          <figure>

          envolve.
          Pharmacy Solutions

          TH

          </figure>


          # CLINICAL POLICY Blinatumomab

          1\. Blincyto Prescribing Information. Thousand Oaks, CA: Amgen, Inc .; March 2018. Available at:
          http://pi.amgen.com/~/media/amgen/repositorysites/pi-amgen-
          com/blincyto/blincyto_pi_hcp_english.ashx. Accessed April 2018.

          2\. National Comprehensive Cancer Network Drugs and Biologics Compendium. Available at nccn.org.
          Accessed April 2018.

          3\. National Comprehensive Cancer Network Guidelines. Acute lymphoblastic leukemia; Version 1.2018.
          Available at nccn.org. Accessed April 2018.

          4\. Clinical Pharmacology [database online]. Tampa, FL: Gold Standard, Inc .; 2018. Available at
          http://www.clinicalpharmacology-ip.com/.

          5\. Gökbuget N, Dombret H, Bonifacio M, et al. Blinatumomab for minimal residual disease in adults with
          B-precursor acute lymphoblastic leukemia. Blood 2018; doi: https://doi.org/10.1182/blood-2017-08-
          798322\.

          6\. Martinelli G, Boissel N, Chevallier P, et al. Complete hematologic and molecular response in adult
          patients with relapsed/refractory Philadelphia chromosome-positive B-precursor acute lymphoblastic
          leukemia following treatment with blinatumomab: results from a phase II, single-arm, multicenter
          study. J Clin Oncol. 2017 Jun 1; 35(16):1795-1802. doi: 10.1200/JCO.2016.69.3531. Epub 2017 Mar
          29\.

          7\. Kim DY, Joo YD, Lim SN, et al. Nilotinib combined with multiagent chemotherapy for newly diagnosed
          Philadelphia-positive acute lymphoblastic leukemia. Blood 2015; 126: 746-756.

          8\. Gambacorti-Passerini C, Kantarjian HM, Kim DW, et al. Longterm efficacy and safety of bosutinib in
          patients with advanced leukemia following resistance/ intolerance to imatinib and other tyrosine
          kinase inhibitors. Am J Hematol 2015; 90:755-768.


          <table>
          <tr>
          <th>Reviews, Revisions, and Approvals</th>
          <th>Date</th>
          <th>P&amp;T Approval Date</th>
          </tr>
          <tr>
          <td>Policy created.</td>
          <td>05.08.18</td>
          <td>08.18</td>
          </tr>
          </table>


          ## Important Reminder

          This clinical policy has been developed by appropriately experienced and licensed health care
          professionals based on a review and consideration of currently available generally accepted standards of
          medical practice; peer-reviewed medical literature; government agency/program approval status;
          evidence-based guidelines and positions of leading national health professional organizations; views of
          physicians practicing in relevant clinical areas affected by this clinical policy; and other available clinical
          information.

          This Clinical Policy is not intended to dictate to providers how to practice medicine, nor does it constitute
          a contract or guarantee regarding payment or results. Providers are expected to exercise professional
          medical judgment in providing the most appropriate care, and are solely responsible for the medical
          advice and treatment of members.

          This policy is the property of Envolve Pharmacy Solutions. Unauthorized copying, use, and distribution of
          this Policy or any information contained herein is strictly prohibited. By accessing this policy, you agree to
          be bound by the foregoing terms and conditions, in addition to the Site Use Agreement for Health Plans
          associated with Envolve Pharmacy Solutions.

          @2018 Envolve Pharmacy Solutions. All rights reserved. All materials are exclusively owned by Envolve
          Pharmacy Solutions and are protected by United States copyright law and international copyright law. No
          part of this publication may be reproduced, copied, modified, distributed, displayed, stored in a retrieval
          system, transmitted in any form or by any means, or otherwise published without the prior written
          permission of Envolve Pharmacy Solutions. You may not alter or remove any trademark, copyright or
          other notice contained herein.

          <!-- PageNumber="Page 4 of 4" -->'''

In [94]:
response = await evaluator.generate_responses(patient_info, physician_info, clinical_info, policy_text)
structured_output = await evaluator._summarize_autodetermination(response)

In [95]:
json.loads(structured_output)

{'Determination': 'Approved',
 'Rationale': 'The prior authorization request for Blinatumomab aligns with the policy criteria for treating B-cell precursor acute lymphoblastic leukemia (B-ALL) in remission but positive for minimal residual disease (MRD+). All necessary criteria outlined in the policy are fully met, including diagnosis, physician qualifications, and dosage limitations.',
 'DetailedAnalysis': {'PolicyCriteriaAssessment': [{'CriterionName': 'Diagnosis of B-cell precursor acute lymphoblastic leukemia (B-ALL)',
    'Assessment': 'Fully Met',
    'Evidence': 'Patient is diagnosed with B-cell precursor acute lymphoblastic leukemia (B-ALL), Philadelphia chromosome-negative, and is MRD-positive.',
    'PolicyReference': 'Diagnosis of B-cell precursor acute lymphoblastic leukemia (B-ALL) (Section I.A.1)'},
   {'CriterionName': 'Prescribed by or in consultation with an oncologist or hematologist',
    'Assessment': 'Fully Met',
    'Evidence': 'The prescribing physician, Dr. Onco

In [99]:
os.environ['AZURE_OPENAI_CHAT_DEPLOYMENT_01']=""
evaluator_4o = AutoDeterminationEvaluator(cases_dir="../../evals/cases", temp_dir="./temp_evaluation_rag", config_file_path="../../../.azure/config.json")
evaluator_4o.auto_determinator = auto_pa_determinator

In [100]:
response = await evaluator_4o.generate_responses(patient_info, physician_info, clinical_info, policy_text)
structured_output = await evaluator_4o._summarize_autodetermination(response)

In [101]:
json.loads(structured_output)

{'Determination': 'Approved',
 'Rationale': 'The prior authorization request for Blinatumomab is aligned with the policy criteria for the treatment of B-cell precursor acute lymphoblastic leukemia (B-ALL) in remission but positive for minimal residual disease (MRD+). All necessary conditions, including diagnosis, prescription by a qualified specialist, and dosage requirements, are fully met.',
 'DetailedAnalysis': {'PolicyCriteriaAssessment': [{'CriterionName': 'Diagnosis of B-cell precursor acute lymphoblastic leukemia (B-ALL)',
    'Assessment': 'Fully Met',
    'Evidence': 'The patient is diagnosed with B-cell precursor acute lymphoblastic leukemia (B-ALL) and is MRD+.',
    'PolicyReference': 'Diagnosis of B-cell precursor acute lymphoblastic leukemia (B-ALL); (Policy/Criteria I.A.1)'},
   {'CriterionName': 'Prescribed by or in consultation with an oncologist or hematologist',
    'Assessment': 'Fully Met',
    'Evidence': 'The prescribing physician, Dr. Oncoso, specializes in Pedi